# Darwin Core Conversion of eDNA Sequence Data From the FAIRe (NOAA Version) metadata template 

**Version:** 3.0

**Authors:** Katherine Silliman, Bayden Willms

**Last Updated:** 9-June-2025

This notebook is for converting a [FAIR-eDNA](https://fair-edna.github.io/index.html)-based data sheet to DarwinCore for submission to OBIS. It has been testing on a Windows 11 laptop, with Python 3.11. 

To generate the input files for edna2obis, please run [FAIRe2NODE](https://github.com/aomlomics/FAIReSheets/tree/FAIRe2NODE) to generate your own FAIR-eDNA (NOAA) template. Once you've filled in your data, you are ready to begin.

This newest version of edna2obis takes the same input files as the ODE, the [Ocean DNA Explorer](https://www.oceandnaexplorer.org/). Explore your data (publically or privately) with visualizations, API capabilities, and more through ODE. 

[FAIR-eDNA NOAA Google Sheet](https://docs.google.com/spreadsheets/d/1mkjfUQW3gTn3ezhMQmFDQn4EBoQ2Xv4SZeSd9sqagoU/edit?gid=0#gid=0)

**Requirements:**
- Python 3
- Python 3 packages:
    - os
- External packages:
    - Bio.Entrez from biopython
    - numpy
    - pandas
    - openpyxl
    - pyworms
    - multiprocess
- Custom modules:
    - WoRMS_matching
    - analysis_helpers

**Resources:**
- Abarenkov K, Andersson AF, Bissett A, Finstad AG, Fossøy F, Grosjean M, Hope M, Jeppesen TS, Kõljalg U, Lundin D, Nilsson RN, Prager M, Provoost P, Schigel D, Suominen S, Svenningsen C & Frøslev TG (2023) Publishing DNA-derived data through biodiversity data platforms, v1.3. Copenhagen: GBIF Secretariat. https://doi.org/10.35035/doc-vf1a-nr22.https://doi.org/10.35035/doc-vf1a-nr22.
- [OBIS manual](https://manual.obis.org/dna_data.html)
- [TDWG Darwin Core Occurrence Core](https://dwc.tdwg.org/terms/#occurrence)
- [GBIF DNA Derived Data Extension](https://tools.gbif.org/dwca-validator/extension.do?id=http://rs.gbif.org/terms/1.0/DNADerivedData)
- https://github.com/iobis/dataset-edna

**Citation**  
Silliman K, Anderson S, Storo R, Thompson L (2023) A Case Study in Sharing Marine eDNA Metabarcoding Data to OBIS. Biodiversity Information Science and Standards 7: e111048. https://doi.org/10.3897/biss.7.111048


## Installation  

```bash
conda create -n edna2obis
conda activate edna2obis
conda install -c conda-forge notebook
conda install -c conda-forge nb_conda_kernels

conda install -c conda-forge numpy pandas
conda install -c conda-forge openpyxl

#worms conversion
conda install -c conda-forge pyworms
conda install -c conda-forge multiprocess
conda install -c conda-forge biopython
```

In [2]:
## Imports
import os

import numpy as np
import pandas as pd

import WoRMS_v3_matching # new custom functions for querying WoRMS API


In [3]:
# jupyter notebook parameters
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.max_columns', 50)

Note that in a Jupyter Notebook, the current directory is always where the .ipynb file is being run.

## Prepare Input Data 

**Project data and metadata**  
This workflow assumes that you have your project metadata in an Excel sheet formatted like the FAIR-eDNA template located **TODO NEED TO ADD CORRECT LINK**[here](https://docs.google.com/spreadsheets/d/1YBXFU9PuMqm7IT1tp0LTxQ1v2j0tlCWFnhSpy-EBwPw/edit?usp=drive_link). Instructions for filling out the metadata template are located in the 'README' sheet and at the [documentation website](https://noaa-omics-templates.readthedocs.io/en/latest/).

**eDNA and taxonomy data**  
The eDNA data and assigned taxonomy should be in a specific tab-delimited format. ![asv_table format](../images/asv_table.png)

This file is generated automatically by [Tourmaline 2](https://github.com/aomlomics/tourmaline/tree/develop), in X location. If your data was generated with Qiime2 or a previous version of Tourmaline, you can convert the `table.qza`, `taxonomy.qza`, and `repseqs.qza` outputs to the correct format using the `create_asv_seq_taxa_obis.sh` shell script.

Example:  

``` bash
#Run this with a qiime2 environment. 
bash create_asv_seq_taxa_obis.sh -f \
../gomecc_v2_raw/table-16S-merge.qza -t ../gomecc_v2_raw/taxonomy-16S-merge.qza -r ../gomecc_v2_raw/repseqs-16S-merge.qza \
-o ../gomecc_v2_raw/gomecc-16S-asv.tsv
```


## Set configs  

Below you can set definitions for parameters used in the code. 

| Parameter           | Description                                                                                                       | Example                                                                                              |
|---------------------|-------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------|
| `sampleMetadata`    | Name of sheet in FAIRe template data Excel file with sample metadata.                                                    | "sampleMetadata"                                                                                  |
| `experimentRunMetadata`         | Name of sheet in FAIRe template data Excel file with data about molecular preparation methods.                           | "experimentRunMetadata"                                                                                 |
| `projectMetadata`        | Name of sheet in FAIRe template data Excel file with metadata about the study.                                           | "projectMetadata"                                                                                         |
| `excel_file`        | Path of the FAIRe data Excel file.                                                                                  | "../raw-v3/FAIRe_noaa-aoml-gomecc4.xlsx"                                                  |
| `FAIRe_NOAA_checklist`        | Path of the FAIRe NOAA Checklist, which contains information on mapping to DarwinCore and the expected files for OBIS submission.                                                                                  | "../raw-v3/FAIRe_NOAA_checklist_v1.0.xlsx"                                                  |
| `datafiles`         | Python dictionary, where keys are the amplicon names and the values are the paths to the cooresponding ASV table. |   See example below to format raw data per analysis  |
| `skip_sample_types` | Python list of sample_category values to skip from OBIS submission, such as controls or blanks.                       | [`negative control`, `positive control`] |
| `skip_columns`      | Python list of columns to ignore when submitting to OBIS.                                                         | [`samp_collect_notes`]                                                                                   |
| `taxonomic_api_source`      | Specify whether you want taxonomic assignment from either WoRMS or GBIF APIs.     |  `WoRMS` | 
<!-- | `analysisMetadata`     | Name of sheet in FAIRe template data Excel file with data about analysis methods.                                        | "analysisMetadata_gomecc4_16s_p1"                                                                                      | -->

In [4]:
# EDIT THIS CELL
# Here, you specify where all of your data is, set a few other parameters, and set parameters related to the taxonomic assignment.

# STEP 1
# Assign sheets from your FAIRe Excel metadata file
# NOTE: Left side is what edna2obis calls that data. Right side is the actual sheetname from your FAIRe Excel metadata file:
params = {}
params['sampleMetadata'] = "sampleMetadata"
params['experimentRunMetadata']= "experimentRunMetadata"
params['projectMetadata'] = "projectMetadata"

params['excel_file'] = "../raw-v3/FAIRe_NOAA_noaa-aoml-gomecc4_SHARING.xlsx"
params['FAIRe_NOAA_checklist'] = "../raw-v3/FAIRe_NOAA_checklist_v1.0.xlsx"
# Be sure to also include the paths for the FAIRe metadata Excel sheet, and the FAIRe NOAA Checklist


# STEP 2
# Assign pathnames for your raw data. Each analysis should have 2 raw data files associated with it.
params['datafiles'] = {
    'gomecc4_18s_p1-6_v2024.10_241122': {
        'taxonomy_file': '../raw-v3/asvTaxaFeatures_gomecc4_18s_p1-6_v2024.10_241122.tsv',
        'occurrence_file': '../raw-v3/table_gomecc4_18s_p1-6_v2024.10_241122.tsv'
    }, 
    'gomecc4_16s_p3-6_v2024.10_241122': {
        'taxonomy_file': '../raw-v3/asvTaxaFeatures_gomecc4_16s_p3-6_v2024.10_241122.tsv',
        'occurrence_file': '../raw-v3/table_gomecc4_16s_p3-6_v2024.10_241122.tsv'
    }, 
    'gomecc4_16s_p1-2_v2024.10_241122': {
        'taxonomy_file': '../raw-v3/asvTaxaFeatures_gomecc4_16s_p1-2_v2024.10_241122.tsv',
        'occurrence_file': '../raw-v3/table_gomecc4_16s_p1-2_v2024.10_241122.tsv'
    },
    # Add other analysis runs here, following the pattern:
    # 'your_analysis_run_name': {
    #     'taxonomy_file': 'path/to/your/asvTaxaFeatures_your_analysis_run_name.tsv',
    #     'occurrence_file': 'path/to/your/table_your_analysis_run_name.tsv'
    # },
}


# STEP 3:
# However you denote control / blank samples, specify here
# 'skip_sample_types' is the column name in your metadata, and negative + positive controls are values in that column
params['skip_sample_types'] = ['negative control','positive control']
params['skip_columns']= ['samp_collect_notes','date_modified','modified_by']


# STEP 4:
# Specify which API you would like to use to assign taxonomy. Options are either WoRMS or GBIF
# Taxonomic Assignment Parameters:
params['taxonomic_api_source'] = 'WoRMS'
# params['taxonomic_api_source'] = 'GBIF'


# STEP 5:
# Define which assays should not consider 'species' rank in their taxonomic assignment.
# This is because certain assays (for example, 16S) species' level assignments are not useful / correct.
# 	Additionally, for example, 18S species' level assignments are good, and we want them!
# This should be the exact 'assay_name' value as found in your analysisMetadata sheets (cell D3)
# and subsequently in the 'assay_name' column of the intermediate occurrence.csv
params['user_defined_assays_to_skip_species'] = [
    'ssu16sv4v5-emp',  # Example, replace with your actual 16S assay names
    # 'another_16s_assay_name_if_any',
]


## Load data

Note that in a Jupyter Notebook, the current directory is always where the .ipynb file is being run.

### Load project, sample, experimentRun, and analysis data from the FAIRe Excel file

projectMetadata, sampleMetadata, and experimentRunMetadata can be loaded normally, but we dynamically load the analysisMetadata sheet(s). The user may have any number of analysisMetadata sheets in their submission, and the cell below will detect each one automatically and load their data. 

In [5]:
# Discover all sheets in the Excel file
excel = pd.ExcelFile(params['excel_file'])
all_sheets = excel.sheet_names

# Find analysis metadata sheets
analysis_sheets = [sheet for sheet in all_sheets if sheet.startswith('analysisMetadata')]

# Load the main data sheets (projectMetadata, sampleMetadata, experimentRunMetadata)
data = pd.read_excel(
    params['excel_file'],
    [params['projectMetadata'], params['sampleMetadata'], params['experimentRunMetadata']],
    index_col=None, na_values=[""], comment="#"
)

# Load all analysis metadata sheets
# This dictionary stores the actual DataFrames from each analysisMetadata sheet.
analysis_data_by_assay = {}

for sheet_name_iter in analysis_sheets:
    # Load the sheet
    analysis_df = pd.read_excel(params['excel_file'], sheet_name_iter)
    
    # Get assay_name and analysis_run_name from specific cells (D3 and D4 in FAIRe template)
    # Ensure these are strings for reliable dictionary keys.
    assay_name = str(analysis_df.iloc[1, 3])        # Corresponds to Excel cell D3
    analysis_run_name = str(analysis_df.iloc[2, 3]) # Corresponds to Excel cell D4
    
    print(f"Processing Excel sheet '{sheet_name_iter}': Found assay '{assay_name}' with analysis run '{analysis_run_name}'")
    
    # Store the analysis DataFrame in analysis_data_by_assay, organized by assay_name then analysis_run_name
    if assay_name not in analysis_data_by_assay:
        analysis_data_by_assay[assay_name] = {}
    analysis_data_by_assay[assay_name][analysis_run_name] = analysis_df

# Add the structured analysis data to the main 'data' dictionary
data['analysis_data_by_assay'] = analysis_data_by_assay

# For backward compatibility or general reference, store the DataFrame of the first analysis sheet found
if analysis_sheets:
    data['analysisMetadata'] = pd.read_excel(params['excel_file'], analysis_sheets[0])
else:
    print("Warning: No analysis metadata sheets found! 'data['analysisMetadata']' will not be populated.")

# Print summary of analyses by assay from the loaded metadata sheets
print("\nSummary of analyses by assay (from analysisMetadata sheets):")
for assay, analyses_dict in analysis_data_by_assay.items():
    print(f"  - Assay '{assay}': {len(analyses_dict)} analysis run(s)")
    for run_name_key in analyses_dict.keys():
        print(f"    - {run_name_key}")

# Verify the user-defined params['analysis_files'] from Cell 8
print("\nSummary of analyses and their corresponding raw data files:")
if 'datafiles' in params and params['datafiles']:
    for run_name, files_dict in params['datafiles'].items():
        print(f"  Analysis Run Name: '{run_name}'")
        print(f"    Taxonomy File: {files_dict.get('taxonomy_file', 'Not specified')}")
        print(f"    Occurrence File: {files_dict.get('occurrence_file', 'Not specified')}")
        # Check if this run_name from params matches one found in the Excel sheets
        found_in_excel = any(run_name in an_dict for an_dict in analysis_data_by_assay.values())
        if not found_in_excel:
            print(f"    Warning: Analysis run name '{run_name}' from params['datafiles'] was not found as an analysis_run_name in any analysisMetadata sheet.")
else:
    print("  params['datafiles'] is empty or not defined in Cell 3.")

Processing Excel sheet 'analysisMetadata_gomecc4_16s_p1': Found assay 'ssu16sv4v5-emp' with analysis run 'gomecc4_16s_p1-2_v2024.10_241122'
Processing Excel sheet 'analysisMetadata_gomecc4_16s_p3': Found assay 'ssu16sv4v5-emp' with analysis run 'gomecc4_16s_p3-6_v2024.10_241122'
Processing Excel sheet 'analysisMetadata_gomecc4_18s_p1': Found assay 'ssu18sv9-emp' with analysis run 'gomecc4_18s_p1-6_v2024.10_241122'

Summary of analyses by assay (from analysisMetadata sheets):
  - Assay 'ssu16sv4v5-emp': 2 analysis run(s)
    - gomecc4_16s_p1-2_v2024.10_241122
    - gomecc4_16s_p3-6_v2024.10_241122
  - Assay 'ssu18sv9-emp': 1 analysis run(s)
    - gomecc4_18s_p1-6_v2024.10_241122

Summary of analyses and their corresponding raw data files:
  Analysis Run Name: 'gomecc4_18s_p1-6_v2024.10_241122'
    Taxonomy File: ../raw-v3/asvTaxaFeatures_gomecc4_18s_p1-6_v2024.10_241122.tsv
    Occurrence File: ../raw-v3/table_gomecc4_18s_p1-6_v2024.10_241122.tsv
  Analysis Run Name: 'gomecc4_16s_p3-6_v

In [6]:
#rename keys in data dictionary to a general term
data['sampleMetadata'] = data.pop(params['sampleMetadata'])
data['experimentRunMetadata'] = data.pop(params['experimentRunMetadata'])
# The line below is already done in Cell 4. It is treated differently because the exact sheet name is not 'analysisMetadata'
# data['analysisMetadata'] = data.pop(params['analysisMetadata'])
data['projectMetadata'] = data.pop(params['projectMetadata'])

#### sampleMetadata 
Contextual data about the samples collected, such as when it was collected, where it was collected from, what kind of sample it is, and what were the properties of the environment or experimental condition from which the sample was taken. Each row is a distinct sample, or Event. Most of this information is recorded during sample collection. This sheet contains terms from the FAIRe NOAA data template. 

In [7]:
data['sampleMetadata'].head()

,samp_name,samp_category,neg_cont_type,pos_cont_type,materialSampleID,sample_derived_from,sample_composed_of,rel_cont_id,biological_rep_relation,decimalLongitude,decimalLatitude,verbatimLongitude,verbatimLatitude,verbatimCoordinateSystem,verbatimSRS,geo_loc_name,eventDate,eventDurationValue,verbatimEventDate,verbatimEventTime,env_broad_scale,env_local_scale,env_medium,habitat_natural_artificial_0_1,samp_collect_method,...,phosphate,phosphate_unit,pressure,pressure_unit,silicate,silicate_unit,tot_alkalinity,tot_alkalinity_unit,transmittance,transmittance_unit,serial_number,line_id,station_id,ctd_cast_number,ctd_bottle_number,replicate_number,extract_id,extract_plate,extract_well_number,extract_well_position,biosample_accession,organism,samp_collect_notes,dna_yield,dna_yield_unit
0,GOMECC4_27N_Sta1_Deep_A,sample,NaN,NaN,GOMECC4_27N_Sta1_Deep,NaN,NaN,NaN,NaN,-79.618,26.997,NaN,NaN,NaN,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,NaN,NaN,NaN,marine biome [ENVO:00000447],marine mesopelagic zone [ENVO:00000213],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,...,1.94489,µmol/kg,623,dbar,20.3569,µmol/kg,2318.9,µmol/kg,4.7221,NaN,GOMECC4_001,27N,Sta1,not provided,3,A,Plate4_52,GOMECC2021_Plate4,52,D7,SAMN37516091,seawater metagenome,DCM = deep chlorophyl max.,12.057,ng
1,GOMECC4_27N_Sta1_Deep_B,sample,NaN,NaN,GOMECC4_27N_Sta1_Deep,NaN,NaN,NaN,NaN,-79.618,26.997,NaN,NaN,NaN,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,NaN,NaN,NaN,marine biome [ENVO:00000447],marine mesopelagic zone [ENVO:00000213],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,...,1.94489,µmol/kg,623,dbar,20.3569,µmol/kg,2318.9,µmol/kg,4.7221,NaN,GOMECC4_002,27N,Sta1,not provided,3,B,Plate4_60,GOMECC2021_Plate4,60,D8,SAMN37516092,seawater metagenome,DCM was around 80 m and not well defined.,17.115,ng
2,GOMECC4_27N_Sta1_Deep_C,sample,NaN,NaN,GOMECC4_27N_Sta1_Deep,NaN,NaN,NaN,NaN,-79.618,26.997,NaN,NaN,NaN,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,NaN,NaN,NaN,marine biome [ENVO:00000447],marine mesopelagic zone [ENVO:00000213],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,...,1.94489,µmol/kg,623,dbar,20.3569,µmol/kg,2318.9,µmol/kg,4.7221,NaN,GOMECC4_003,27N,Sta1,not provided,3,C,Plate4_62,GOMECC2021_Plate4,62,F8,SAMN37516093,seawater metagenome,Surface CTD bottles did not fire correctly; hand niskin bottle used for the surface cast. PM cast.,10.8345,ng
3,GOMECC4_27N_Sta1_DCM_A,sample,NaN,NaN,GOMECC4_27N_Sta1_DCM,NaN,NaN,NaN,NaN,-79.618,26.997,NaN,NaN,NaN,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,NaN,NaN,NaN,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,...,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,NaN,GOMECC4_004,27N,Sta1,not provided,14,A,Plate4_53,GOMECC2021_Plate4,53,E7,SAMN37516094,seawater metagenome,Only enough water for 2 surface replicates.,223.5,ng
4,GOMECC4_27N_Sta1_DCM_B,sample,NaN,NaN,GOMECC4_27N_Sta1_DCM,NaN,NaN,NaN,NaN,-79.618,26.997,NaN,NaN,NaN,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,NaN,NaN,NaN,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,...,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,NaN,GOMECC4_005,27N,Sta1,not provided,14,B,Plate4_46,GOMECC2021_Plate4,46,F6,SAMN37516095,seawater metagenome,NaN,103.26,ng


#### experimentRunMetadata  
Contextual data about how the samples were prepared for sequencing. Includes how they were extracted, what amplicon was targeted, how they were sequenced. Each row is a separate sequencing library preparation, distinguished by a unique lib_id. **TODO: MIGHT NEED HELP WITH THIS DESCRIPTION**

In [8]:
data['experimentRunMetadata'].head(2)

,samp_name,assay_name,pcr_plate_id,lib_id,seq_run_id,lib_conc,lib_conc_unit,lib_conc_meth,phix_perc,mid_forward,mid_reverse,filename,filename2,checksum_filename,checksum_filename2,associatedSequences,input_read_count
0,GOMECC4_NegativeControl_1,ssu16sv4v5-emp,not applicable,GOMECC16S_Neg1,20220613_Amplicon_PE250,NaN,NaN,NaN,NaN,TAGCAGCT,CGTCGCTA,GOMECC16S_Neg1_S499_L001_R1_001.fastq.gz,GOMECC16S_Neg1_S499_L001_R2_001.fastq.gz,NaN,NaN,https://www.ncbi.nlm.nih.gov/sra/SRR26148505 | https://www.ncbi.nlm.nih.gov/biosample/SAMN37516589 | https://www.ncbi.nlm.nih.gov/bioproject/PRJNA...,29319
1,GOMECC4_NegativeControl_2,ssu16sv4v5-emp,not applicable,GOMECC16S_Neg2,20220613_Amplicon_PE250,NaN,NaN,NaN,NaN,TAGCAGCT,CTAGAGCT,GOMECC16S_Neg2_S500_L001_R1_001.fastq.gz,GOMECC16S_Neg2_S500_L001_R2_001.fastq.gz,NaN,NaN,https://www.ncbi.nlm.nih.gov/sra/SRR26148503 | https://www.ncbi.nlm.nih.gov/biosample/SAMN37516590 | https://www.ncbi.nlm.nih.gov/bioproject/PRJNA...,30829


### Load ASV data  
There is one ASV file for each marker that was sequenced. The ASV data files have one row for each unique amplicon sequence variants (ASVs). They contain the ASV DNA sequence, a unique hash identifier the taxonomic assignment for each ASV, the confidence given that assignment by the naive-bayes classifier, and then the number of reads observed in each sample. 

This file is created automatically with [Tourmaline v.2023.5+](https://github.com/aomlomics/tourmaline), and is found in `01-taxonomy/asv_taxa_sample_table.tsv`. 

| column name    | definition                                                                                                                                                                                                                                                                                                                                                                                              |
|----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| featureid      | A hash of the ASV sequence, used as a unique identifier for the ASV.                                                                                                                                                                                                                                                                                                                                    |
| taxonomy       | The full taxonomy assigned to an ASV sequence. This string could be formatted in very different ways depending on the reference database used during classification, however it should always be in reverse rank order separated by ;. We provide examples for how to process results from a Silva classifier and the PR2 18S classifier. For other taxonomy formats, the code will need to be adapted. |
| Confidence     | This is the confidence score assigned the taxonomic classification with a naive-bayes classifier.                                                                                                                                                                                                                                                                                                       |
| sample columns | The next columns each represent a sample (or eventID), and the number of reads for that ASV observed in the sample.                                                                                                                                                                                                                                                                                     |

In [9]:
# Read in ASV tables for each analysis run.
# We now have separate taxonomy and occurrence files per analysis_run_name.

raw_data_tables = {} # This will store DataFrames for taxonomy and occurrences for each run

print("Loading raw data into dataframes:")
if 'datafiles' in params and params['datafiles']:
    for analysis_run_name, file_paths in params['datafiles'].items():
        print(f"  Processing analysis run: {analysis_run_name}")
        raw_data_tables[analysis_run_name] = {}
        
        # Load taxonomy file
        if 'taxonomy_file' in file_paths:
            tax_path = file_paths['taxonomy_file']
            try:
                raw_data_tables[analysis_run_name]['taxonomy'] = pd.read_table(tax_path, sep='\t', low_memory=False)
                print(f"    Successfully loaded taxonomy file: {tax_path}")
                # Optional: print shape or head if useful for verification
                # print(f"      Shape: {raw_data_tables[analysis_run_name]['taxonomy'].shape}")
            except FileNotFoundError:
                print(f"    ERROR: Taxonomy file not found at {tax_path}")
            except Exception as e:
                print(f"    ERROR: Could not load taxonomy file {tax_path}. Error: {e}")
        else:
            print(f"    Warning: Taxonomy file path not specified for {analysis_run_name}")
            
        # Load occurrence file
        if 'occurrence_file' in file_paths:
            occ_path = file_paths['occurrence_file']
            try:
                # Row 1 is a comment, Row 2 is header, Data starts Row 3.
                # skiprows=1 means skip the first row (the comment).
                # header=0 (after skipping 1 row) means use the NEW first row (original Row 2) as headers.
                df_occ = pd.read_table(occ_path, 
                                       sep='\t', 
                                       skiprows=1,  # Skip the comment line (original Row 1)
                                       header=0,    # Use the next line (original Row 2) as column headers
                                       low_memory=False)

                # It's common for TSV tools to export the first column header with a '#' prefix, e.g., "#OTU ID".
                # If so, remove the leading '#'.
                if df_occ.columns[0].startswith('#'):
                    df_occ.rename(columns={df_occ.columns[0]: df_occ.columns[0][1:]}, inplace=True)
                
                raw_data_tables[analysis_run_name]['occurrence'] = df_occ
                print(f"    Successfully loaded occurrence file: {occ_path}")
            except FileNotFoundError:
                print(f"    ERROR: Occurrence file not found at {occ_path}")
            except Exception as e:
                print(f"    ERROR: Could not load occurrence file {occ_path}. Error: {e}")
        else:
            print(f"    Warning: Occurrence file path not specified for {analysis_run_name}")

# EXAMPLE: to access the taxonomy DataFrame for 'gomecc4_18s_p1-6_v2024.10_241122':
# raw_data_tables['gomecc4_18s_p1-6_v2024.10_241122']['taxonomy']
# And its occurrence DataFrame:
# raw_data_tables['gomecc4_18s_p1-6_v2024.10_241122']['occurrence']


Loading raw data into dataframes:
  Processing analysis run: gomecc4_18s_p1-6_v2024.10_241122
    Successfully loaded taxonomy file: ../raw-v3/asvTaxaFeatures_gomecc4_18s_p1-6_v2024.10_241122.tsv
    Successfully loaded occurrence file: ../raw-v3/table_gomecc4_18s_p1-6_v2024.10_241122.tsv
  Processing analysis run: gomecc4_16s_p3-6_v2024.10_241122
    Successfully loaded taxonomy file: ../raw-v3/asvTaxaFeatures_gomecc4_16s_p3-6_v2024.10_241122.tsv
    Successfully loaded occurrence file: ../raw-v3/table_gomecc4_16s_p3-6_v2024.10_241122.tsv
  Processing analysis run: gomecc4_16s_p1-2_v2024.10_241122
    Successfully loaded taxonomy file: ../raw-v3/asvTaxaFeatures_gomecc4_16s_p1-2_v2024.10_241122.tsv
    Successfully loaded occurrence file: ../raw-v3/table_gomecc4_16s_p1-2_v2024.10_241122.tsv


OPTIONAL: Verify the shapes of the raw data before OBIS/GBIF conversion takes place. Does the number of rows and columns match your raw data?

In [10]:
# Inspect the keys of the newly loaded raw_data_tables
# This will show the analysis_run_names for which data was loaded.
if 'raw_data_tables' in locals() and raw_data_tables: # Check if it exists and is not empty
    print("Analysis runs for which data was loaded:")
    for run_name in raw_data_tables.keys():
        print(f"  - {run_name}")
        if 'taxonomy' in raw_data_tables[run_name]:
            print(f"    - Taxonomy table shape: {raw_data_tables[run_name]['taxonomy'].shape}")
        else:
            print(f"    - Taxonomy table: Not loaded or error during load.")
        if 'occurrence' in raw_data_tables[run_name]:
            print(f"    - Occurrence table shape: {raw_data_tables[run_name]['occurrence'].shape}")
        else:
            print(f"    - Occurrence table: Not loaded or error during load.")
else:
    print("raw_data_tables dictionary is not defined or is empty.")

# Example of how to access a specific table\:
# print(raw_data_tables['gomecc4_18s_p1-6_v2024.10_241122']['taxonomy'].head(2))
# print(raw_data_tables['gomecc4_18s_p1-6_v2024.10_241122']['occurrence'].head(2))

Analysis runs for which data was loaded:
  - gomecc4_18s_p1-6_v2024.10_241122
    - Taxonomy table shape: (24473, 14)
    - Occurrence table shape: (24473, 501)
  - gomecc4_16s_p3-6_v2024.10_241122
    - Taxonomy table shape: (49523, 12)
    - Occurrence table shape: (49523, 312)
  - gomecc4_16s_p1-2_v2024.10_241122
    - Taxonomy table shape: (19540, 12)
    - Occurrence table shape: (19540, 198)


In [11]:
# Choose an analysis run name that you have defined in params['datafiles']
# For example, the first one:
analysis_to_inspect = list(params['datafiles'].keys())[0] if params['datafiles'] else None

if analysis_to_inspect and analysis_to_inspect in raw_data_tables:
    if 'occurrence' in raw_data_tables[analysis_to_inspect]:
        print(f"Head of OCCURRENCE table for '{analysis_to_inspect}':")
        display(raw_data_tables[analysis_to_inspect]['occurrence'].iloc[:, 0:20].head())
    else:
        print(f"Occurrence table not found for '{analysis_to_inspect}'.")
else:
    if not analysis_to_inspect:
        print("params['datafiles'] is empty. No analysis run to inspect.")
    else:
        print(f"Analysis run '{analysis_to_inspect}' not found in loaded raw_data_tables.")

Head of OCCURRENCE table for 'gomecc4_18s_p1-6_v2024.10_241122':


,OTU ID,GOMECC4_27N_Sta1_DCM_A,GOMECC4_27N_Sta1_DCM_B,GOMECC4_27N_Sta1_DCM_C,GOMECC4_27N_Sta1_Deep_A,GOMECC4_27N_Sta1_Deep_B,GOMECC4_27N_Sta1_Deep_C,GOMECC4_27N_Sta1_Surface_A,GOMECC4_27N_Sta1_Surface_B,GOMECC4_27N_Sta4_DCM_A,GOMECC4_27N_Sta4_DCM_B,GOMECC4_27N_Sta4_DCM_C,GOMECC4_27N_Sta4_Deep_A,GOMECC4_27N_Sta4_Deep_B,GOMECC4_27N_Sta4_Deep_C,GOMECC4_27N_Sta4_Surface_A,GOMECC4_27N_Sta4_Surface_B,GOMECC4_27N_Sta4_Surface_C,GOMECC4_27N_Sta6_DCM_A,GOMECC4_27N_Sta6_DCM_B
0,36aa75f9b28f5f831c2d631ba65c2bcb,1518.0,0.0,0.0,6.0,0.0,0.0,0.0,4268.0,2002.0,0.0,14.0,0.0,0.0,0.0,9532.0,1930.0,2037.0,0.0,0.0
1,4e38e8ced9070952b314e1880bede1ca,963.0,316.0,543.0,19.0,10.0,0.0,0.0,0.0,613.0,561.0,434.0,0.0,395.0,297.0,76.0,915.0,1447.0,140.0,0.0
2,5d4df37251121c08397c6fbc27b06175,0.0,4.0,0.0,12.0,5.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,0.0,864.0,409.0
3,f863f671a575c6ab587e8de0190d3335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,92.0,2672.0,2424.0,2605.0,1918.0
4,2a31e5c01634165da99e7381279baa75,1165.0,2267.0,2206.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1103.0,157.0,0.0,104.0,941.0


### Drop samples with unwanted sample types  

Often with eDNA projects, we have control samples that are sequenced along with our survey samples. These can include filtering distilled water, using pure water instead of DNA in a PCR or DNA extraction protocol, or a mock community of known microbial taxa. Controls can help identify and mitigate contaminant DNA in our samples, but are not useful for biodiversity platforms like OBIS. You can select which sample_type values to drop with the `skip_sample_types` parameter.

In [12]:
# This should be your Cell 23 (or equivalent)
samps_to_remove = data['sampleMetadata']['samp_category'].isin(params['skip_sample_types'])
samples_to_drop = data['sampleMetadata']['samp_name'][samps_to_remove].astype(str).str.strip().tolist() 

You can view the list of samples to be dropped below.

In [13]:
samples_to_drop

['GOMECC4_Blank_DIW_20210915_A',
 'GOMECC4_Blank_DIW_20210915_B',
 'GOMECC4_Blank_DIW_20210915_C',
 'GOMECC4_Blank_DIW_20210930_A',
 'GOMECC4_Blank_DIW_20210930_B',
 'GOMECC4_Blank_DIW_20210930_C',
 'GOMECC4_Blank_DIW_20211011_A',
 'GOMECC4_Blank_DIW_20211011_B',
 'GOMECC4_Blank_DIW_20211011_C',
 'GOMECC4_Blank_DIW_20211016_A',
 'GOMECC4_Blank_DIW_20211016_B',
 'GOMECC4_Blank_DIW_20211016_C',
 'GOMECC4_ExtractionBlank_1',
 'GOMECC4_ExtractionBlank_11',
 'GOMECC4_ExtractionBlank_12',
 'GOMECC4_ExtractionBlank_3',
 'GOMECC4_ExtractionBlank_5',
 'GOMECC4_ExtractionBlank_7',
 'GOMECC4_ExtractionBlank_9',
 'GOMECC4_MSUControl_1',
 'GOMECC4_MSUControl_2',
 'GOMECC4_MSUControl_3',
 'GOMECC4_MSUControl_4',
 'GOMECC4_MSUControl_5',
 'GOMECC4_MSUControl_6',
 'GOMECC4_MSUControl_7',
 'GOMECC4_NegativeControl_1',
 'GOMECC4_NegativeControl_2',
 'GOMECC4_PositiveControl_1',
 'GOMECC4_PositiveControl_2']

In [14]:
# remove samples from sampleMetadata sheet
data['sampleMetadata'] = data['sampleMetadata'][~samps_to_remove]

In [15]:
# check the samp_category values left in your sampleMetadata. We only want 'sample' (indicating it is not a control or blank).
data['sampleMetadata']['samp_category'].unique()

array(['sample'], dtype=object)

In [16]:
# remove samples from experimentRunMetadata
prep_samps_to_remove = data['experimentRunMetadata']['samp_name'].isin(samples_to_drop)
data['experimentRunMetadata'] = data['experimentRunMetadata'][~prep_samps_to_remove]

##### Drop unwanted samples from ASV files


In [17]:
# This cell REMOVES blank/control samples from the ALREADY LOADED raw data / abundance tables

print("Attempting to remove blank/control samples from loaded occurrence tables...")
if 'raw_data_tables' in locals() and raw_data_tables:
    if 'samples_to_drop' in locals() and samples_to_drop:
        for analysis_run_name, tables_dict in raw_data_tables.items():
            if 'occurrence' in tables_dict and not tables_dict['occurrence'].empty:
                occ_df = tables_dict['occurrence'] # Get a reference to the occurrence DataFrame
                original_cols_count = len(occ_df.columns)
                
                # Identify which of the samples_to_drop are actual columns in THIS occurrence table
                cols_to_remove_in_this_df = [col for col in samples_to_drop if col in occ_df.columns]
                
                if cols_to_remove_in_this_df:
                    # Perform the drop operation. This MODIFIES the DataFrame in raw_data_tables.
                    raw_data_tables[analysis_run_name]['occurrence'] = occ_df.drop(columns=cols_to_remove_in_this_df)
                    
                    print(f"  For analysis run '{analysis_run_name}':")
                    print(f"    Original columns: {original_cols_count}, Columns after removal: {len(raw_data_tables[analysis_run_name]['occurrence'].columns)}")
                    print(f"    Removed ({len(cols_to_remove_in_this_df)} total for this run): {cols_to_remove_in_this_df}") # Print all removed
                else:
                    print(f"  For analysis run '{analysis_run_name}': No specified blank/control samples found to remove in this table's columns.")
            else:
                print(f"  Skipping analysis run '{analysis_run_name}': No 'occurrence' table found or it is empty.")
        print("Finished blank/control sample removal process.")
    else:
        print("WARNING: 'samples_to_drop' list not found or empty. No columns removed from occurrence tables.")
else:
    print("WARNING: 'raw_data_tables' not found or empty. Cannot remove columns.")

Attempting to remove blank/control samples from loaded occurrence tables...
  For analysis run 'gomecc4_18s_p1-6_v2024.10_241122':
    Original columns: 501, Columns after removal: 473
    Removed (28 total for this run): ['GOMECC4_Blank_DIW_20210915_A', 'GOMECC4_Blank_DIW_20210915_B', 'GOMECC4_Blank_DIW_20210915_C', 'GOMECC4_Blank_DIW_20210930_A', 'GOMECC4_Blank_DIW_20210930_B', 'GOMECC4_Blank_DIW_20210930_C', 'GOMECC4_Blank_DIW_20211011_A', 'GOMECC4_Blank_DIW_20211011_B', 'GOMECC4_Blank_DIW_20211011_C', 'GOMECC4_Blank_DIW_20211016_A', 'GOMECC4_Blank_DIW_20211016_B', 'GOMECC4_Blank_DIW_20211016_C', 'GOMECC4_ExtractionBlank_1', 'GOMECC4_ExtractionBlank_11', 'GOMECC4_ExtractionBlank_12', 'GOMECC4_ExtractionBlank_3', 'GOMECC4_ExtractionBlank_5', 'GOMECC4_ExtractionBlank_7', 'GOMECC4_ExtractionBlank_9', 'GOMECC4_MSUControl_1', 'GOMECC4_MSUControl_2', 'GOMECC4_MSUControl_3', 'GOMECC4_MSUControl_4', 'GOMECC4_MSUControl_5', 'GOMECC4_MSUControl_6', 'GOMECC4_MSUControl_7', 'GOMECC4_NegativeCon

### Drop columns with all NAs  

If your project data file has columns with only NAs, this code will check for those, provide their column headers for verification, then remove them.

In [18]:
dropped = pd.DataFrame()

for sheet_name in ['sampleMetadata', 'experimentRunMetadata']:
    # Safety check: ensure the sheet exists in data and is not empty
    if sheet_name in data and not data[sheet_name].empty:
        all_na_cols = data[sheet_name].columns[data[sheet_name].isnull().all(axis=0)]
        res = pd.Series(all_na_cols, name=sheet_name)
        dropped = pd.concat([dropped, res], axis=1)
    elif sheet_name not in data:
        print(f"FYI: Sheet '{sheet_name}' not found in 'data' dictionary. Cannot check for all-NA columns.")
    else: # In data but empty
        print(f"FYI: Sheet '{sheet_name}' is empty. Cannot check for all-NA columns.")
    

Which columns in each sheet have only NA values?

In [19]:
dropped

,sampleMetadata,experimentRunMetadata
0,neg_cont_type,lib_conc
1,pos_cont_type,lib_conc_unit
2,sample_derived_from,lib_conc_meth
3,sample_composed_of,phix_perc
4,rel_cont_id,checksum_filename
...,...,...
71,org_matter,NaN
72,org_matter_unit,NaN
73,org_nitro,NaN
74,org_nitro_unit,NaN


If you are fine with leaving these columns out, proceed:

In [20]:
# Drops all-NA columns from 'sampleMetadata' and 'experimentRunMetadata'
# based on the 'dropped' DataFrame.

sheets_to_clean = ['sampleMetadata', 'experimentRunMetadata']

for sheet_name in sheets_to_clean:
    # Check if 'dropped' has a column for this sheet AND if that column lists any actual columns to drop
    if sheet_name in dropped.columns and not dropped[sheet_name].dropna().empty:
        cols_to_drop = list(dropped[sheet_name].dropna())
        
        if sheet_name in data:
            print(f"Dropping from data['{sheet_name}']: {cols_to_drop}")
            data[sheet_name].drop(columns=cols_to_drop, inplace=True, errors='ignore')

Dropping from data['sampleMetadata']: ['neg_cont_type', 'pos_cont_type', 'sample_derived_from', 'sample_composed_of', 'rel_cont_id', 'biological_rep_relation', 'verbatimLongitude', 'verbatimLatitude', 'verbatimCoordinateSystem', 'eventDurationValue', 'verbatimEventDate', 'verbatimEventTime', 'samp_store_method_additional', 'stationed_sample_dur', 'pump_flow_rate', 'pump_flow_rate_unit', 'prefilter_material', 'filter_diameter', 'filter_surface_area', 'prepped_samp_store_temp', 'prepped_samp_store_sol', 'prepped_samp_store_dur', 'prep_method_additional', 'date_ext', 'nucl_acid_ext_modify', 'dna_cleanup_0_1', 'dna_cleanup_method', 'concentration_method', 'ratioOfAbsorbance260_280', 'pool_dna_num', 'nucl_acid_ext_method_additional', 'samp_weather', 'elev', 'light_intensity', 'suspend_part_matter', 'tidal_stage', 'turbidity', 'water_current', 'solar_irradiance', 'wind_direction', 'wind_speed', 'diss_inorg_nitro', 'diss_inorg_nitro_unit', 'diss_org_carb', 'diss_org_carb_unit', 'diss_org_nitr

### Now lets drop NA rows of each analysisMetadata sheet

In [21]:
# Cell to identify and report rows with empty 'values' in analysisMetadata sheets

analysis_rows_to_drop_info = {} 
expected_value_col_name = 'values' # As per your Excel sheet (column D header)

print(f"Identifying rows with empty '{expected_value_col_name}' column in analysisMetadata sheets...")

if 'analysis_data_by_assay' in data and data['analysis_data_by_assay']:
    for assay_name, analyses_dict in data['analysis_data_by_assay'].items():
        if not isinstance(analyses_dict, dict): continue
        for run_name, analysis_df in analyses_dict.items():
            if not isinstance(analysis_df, pd.DataFrame) or analysis_df.empty: continue

            # Ensure the 'values' column exists
            if expected_value_col_name not in analysis_df.columns:
                print(f"  Warning: Column '{expected_value_col_name}' not found in Assay: '{assay_name}', Run: '{run_name}'. Skipping.")
                continue
            
            # Identify rows where the 'values' column is NA
            empty_values_mask = analysis_df[expected_value_col_name].isnull()
            rows_to_drop_indices = analysis_df[empty_values_mask].index.tolist()

            if rows_to_drop_indices:
                # Get the list of term names for the empty rows
                terms_to_drop = analysis_df.loc[rows_to_drop_indices, 'term_name'].tolist() if 'term_name' in analysis_df.columns else []
                
                # This is the important part: the logic to save the indices for the next cell remains unchanged
                analysis_rows_to_drop_info[(assay_name, run_name)] = rows_to_drop_indices

                # New, cleaner printing logic
                print(f"\nFor Analysis (Assay: '{assay_name}', Run: '{run_name}'), found {len(terms_to_drop)} empty rows:")
                print(f"  {terms_to_drop}")


if not analysis_rows_to_drop_info:
    print(f"\nNo rows with an empty '{expected_value_col_name}' column were found in any analysisMetadata sheet.")
else:
    print("Summary: The terms listed above will be dropped from their respective sheets.")
    print("Proceed to the next cell to finalize their removal.")

Identifying rows with empty 'values' column in analysisMetadata sheets...

For Analysis (Assay: 'ssu16sv4v5-emp', Run: 'gomecc4_16s_p1-2_v2024.10_241122'), found 36 empty rows:
  ['screen_contam_method', 'screen_geograph_method', 'screen_nontarget_method', 'screen_other', 'bioinfo_method_additional', 'output_read_count', 'output_otu_num', 'otu_num_tax_assigned', 'discard_untrimmed', 'qiime2_version', 'tourmaline_asv_method', 'dada2_trunc_len_f', 'dada2pe_trunc_len_r', 'dada2_trim_left_f', 'dada2pe_trim_left_r', 'dada2_max_ee_f', 'dada2pe_max_ee_r', 'dada2_trunc_q', 'dada2_pooling_method', 'dada2_chimera_method', 'dada2_min_fold_parent_over_abundance', 'dada2_n_reads_learn', 'deblur_trim_length', 'deblur_mean_error', 'deblur_indel_prob', 'deblur_indel_max', 'deblur_min_reads', 'deblur_min_size', 'repseqs_min_abundance', 'repseqs_min_length', 'repseqs_max_length', 'repseqs_min_prevalence', 'skl_confidence', 'min_consensus', 'tourmaline_classify_method', 'blca_confidence']

For Analysis (

### If you are okay with those rows being deleted, continue:

In [22]:
# Cell to perform the deletion of rows with empty 'values' from analysisMetadata sheets

if 'analysis_rows_to_drop_info' in locals() and analysis_rows_to_drop_info:
    print(f"\nRemoving identified rows with empty 'values' from analysisMetadata sheets...")
    for (assay_name, run_name), indices_to_drop in analysis_rows_to_drop_info.items():
        if indices_to_drop: 
            try:
                original_count = len(data['analysis_data_by_assay'][assay_name][run_name])
                data['analysis_data_by_assay'][assay_name][run_name].drop(index=indices_to_drop, inplace=True)
                new_count = len(data['analysis_data_by_assay'][assay_name][run_name])
                print(f"  For Assay: '{assay_name}', Run: '{run_name}': Removed {original_count - new_count} row(s).")
            except Exception as e:
                print(f"  An error occurred while dropping rows for Assay: '{assay_name}', Run: '{run_name}': {e}")
    print("\nFinished removing rows with empty 'values' from analysisMetadata sheets.")
    # del analysis_rows_to_drop_info # Optional: clear the info variable
else:
    print("\nNo rows with empty 'values' were previously identified for deletion from analysisMetadata sheets, or 'analysis_rows_to_drop_info' not found.")


Removing identified rows with empty 'values' from analysisMetadata sheets...
  For Assay: 'ssu16sv4v5-emp', Run: 'gomecc4_16s_p1-2_v2024.10_241122': Removed 36 row(s).
  For Assay: 'ssu16sv4v5-emp', Run: 'gomecc4_16s_p3-6_v2024.10_241122': Removed 36 row(s).
  For Assay: 'ssu18sv9-emp', Run: 'gomecc4_18s_p1-6_v2024.10_241122': Removed 37 row(s).

Finished removing rows with empty 'values' from analysisMetadata sheets.


### Now lets check for columns or rows with SOME missing values. OBIS wants those deleted as well

Now let's check which columns have missing values in some of the rows. These should be filled in on the Excel sheet with the appropriate term ('not applicable', 'missing', or 'not collected'). Alternatively, you can drop the column if it is not needed for submission to OBIS.

In [23]:
some = pd.DataFrame()

# Sheets to check for columns with *some* NAs.
sheets_to_examine_for_some_na = ['sampleMetadata', 'experimentRunMetadata']

for sheet_name in sheets_to_examine_for_some_na:
    if sheet_name in data and not data[sheet_name].empty:
        cols_with_some_na = data[sheet_name].columns[data[sheet_name].isnull().any(axis=0)]
        res = pd.Series(cols_with_some_na.tolist(), name=sheet_name) # .tolist() for cleaner Series
        some = pd.concat([some, res], axis=1)

print("Columns with some NAs:")
some

Columns with some NAs:


,sampleMetadata,experimentRunMetadata
0,ph_meth,NaN
1,carbonate_unit,NaN
2,pco2_unit,NaN
3,samp_collect_notes,NaN


Here I'm going to drop all the columns with some missing data, as I don't need them for submission to OBIS.

In [24]:
sheets_to_clean = ['sampleMetadata', 'experimentRunMetadata']

for sheet_name in sheets_to_clean:
    if sheet_name in data and not data[sheet_name].empty: # Check if DataFrame exists and is not empty
        original_columns = data[sheet_name].columns.tolist() # Get column names before dropping
        
        data[sheet_name].dropna(axis=1, how='any', inplace=True)
        
        current_columns = data[sheet_name].columns.tolist() # Get column names after dropping
        dropped_column_names = [col for col in original_columns if col not in current_columns]
        
        if dropped_column_names:
            print(f"From data['{sheet_name}']: Dropped columns: {dropped_column_names}")
        else: # Optional: if no columns were dropped
            print(f"No columns dropped from data['{sheet_name}']")
    elif sheet_name not in data:
        print(f"FYI: Sheet '{sheet_name}' not found in 'data'. No columns dropped.")
    else: # Sheet is in data but empty
        print(f"FYI: Sheet '{sheet_name}' is empty. No columns dropped.")

From data['sampleMetadata']: Dropped columns: ['ph_meth', 'carbonate_unit', 'pco2_unit', 'samp_collect_notes']
No columns dropped from data['experimentRunMetadata']


### Load data dictionary Excel file 
This FAIRe NOAA Checklist Excel file also contains columns for mapping FAIRe fields to the appropriate Darwin Core terms which OBIS is expecting. Currently, we are only preparing an Occurrence core file and a DNA-derived extension file, with Event information in the Occurrence file. Future versions of this workflow will prepare an extendedMeasurementOrFact file as well.

In [25]:
dwc_data = {}
checklist_df = pd.DataFrame()

try:
    checklist_df = pd.read_excel(
        params['FAIRe_NOAA_checklist'], # Ensure this param is set in your params cell
        sheet_name='checklist',
        na_values=[""]
    )
except Exception as e:
    print(f"Error loading 'checklist' sheet: {e}")

# Define relevant column names from your checklist
col_faire_term = 'term_name'
col_output_spec = 'edna2obis_output_file'
col_dwc_mapping = 'dwc_term'

occurrence_maps = []
dna_derived_maps = []

# Process the checklist if it loaded successfully and has the required columns
if not checklist_df.empty and all(col in checklist_df.columns for col in [col_faire_term, col_output_spec, col_dwc_mapping]):
    for _, row in checklist_df.iterrows():
        faire_term = row[col_faire_term]
        output_file = str(row[col_output_spec]).lower() # Convert to string and lowercase
        dwc_term = row[col_dwc_mapping]

        # Add to lists if terms are valid
        if pd.notna(faire_term) and pd.notna(dwc_term) and str(faire_term).strip() and str(dwc_term).strip():
            if 'occurrence' in output_file:
                occurrence_maps.append({'DwC_term': dwc_term, 'FAIRe_term': faire_term})
            if 'dnaderived' in output_file:
                dna_derived_maps.append({'DwC_term': dwc_term, 'FAIRe_term': faire_term})
    
    # Create DataFrames, using DwC_term as index
    dwc_data['occurrence'] = pd.DataFrame(occurrence_maps).drop_duplicates().set_index('DwC_term') if occurrence_maps else \
                             pd.DataFrame(columns=['FAIRe_term']).set_index(pd.Index([], name='DwC_term'))
    
    dwc_data['dnaDerived'] = pd.DataFrame(dna_derived_maps).drop_duplicates().set_index('DwC_term') if dna_derived_maps else \
                             pd.DataFrame(columns=['FAIRe_term']).set_index(pd.Index([], name='DwC_term'))
else:
    # If checklist is empty or missing columns, create empty structures for dwc_data
    if checklist_df.empty and 'FAIRe_NOAA_checklist_excel' in params: # Avoid double error message if file load failed
        print(f"Checklist DataFrame is empty or required columns are missing. Creating empty DwC mappings.")
    dwc_data['occurrence'] = pd.DataFrame(columns=['FAIRe_term']).set_index(pd.Index([], name='DwC_term'))
    dwc_data['dnaDerived'] = pd.DataFrame(columns=['FAIRe_term']).set_index(pd.Index([], name='DwC_term'))

# Print summary
print(f"Darwin Core term mapping created: \n   Occurrence Core mappings: {len(dwc_data['occurrence'])} \n   DNA Derived Extension mappings: {len(dwc_data['dnaDerived'])}")

Darwin Core term mapping created: 
   Occurrence Core mappings: 26 
   DNA Derived Extension mappings: 27


In [26]:
# Print the mapping for the Occurrence Core
# Keep in mind, some terms are hard coded later in this workflow, or are derived by more than one FAIRe term
# This means there may be less fields listed below than what the Occurrence Core will have upon completion of edna2obis
print("Darwin Core Occurrence Mappings:")
print("=" * 50)
occurrence_display = dwc_data['occurrence'].reset_index()
print(occurrence_display.to_string(index=False))

Darwin Core Occurrence Mappings:
              DwC_term             FAIRe_term
            recordedBy             recordedBy
             datasetID             project_id
         parentEventID              samp_name
      materialSampleID       materialSampleID
      decimalLongitude       decimalLongitude
       decimalLatitude        decimalLatitude
         geodeticDatum            verbatimSRS
              locality           geo_loc_name
             eventDate              eventDate
       sampleSizeValue              samp_size
        sampleSizeUnit         samp_size_unit
               eventID                 lib_id
   associatedSequences    associatedSequences
               kingdom                kingdom
                phylum                 phylum
                 class                  class
                 order                  order
                family                 family
                 genus                  genus
        scientificName         scientificName
 

In [27]:
# Print the mapping for the DNA Derived Extension
# Keep in mind, some terms are hard coded later in this workflow, or are derived by more than one FAIRe term
# This means there may be less fields listed below than what the DNA Derived Extension will have upon completion of edna2obis
print("\n\nDNA Derived Data Mappings:")
print("=" * 50)
dna_display = dwc_data['dnaDerived'].reset_index()
print(dna_display.to_string(index=False))



DNA Derived Data Mappings:
               DwC_term                   FAIRe_term
        env_broad_scale              env_broad_scale
        env_local_scale              env_local_scale
             env_medium                   env_medium
    samp_collect_device          samp_collect_device
       samp_mat_process             samp_mat_process
              size_frac                    size_frac
    samp_vol_we_dna_ext          samp_vol_we_dna_ext
          nucl_acid_ext                nucl_acid_ext
          concentration                concentration
      concentrationUnit           concentration_unit
            target_gene                  target_gene
     target_subfragment           target_subfragment
           ampliconSize                 ampliconSize
     pcr_primer_forward           pcr_primer_forward
     pcr_primer_reverse           pcr_primer_reverse
pcr_primer_name_forward      pcr_primer_name_forward
pcr_primer_name_reverse      pcr_primer_name_reverse
   pcr_primer_ref

## Convert to Occurrence file
In order to link the DNA-derived extension metadata to our OBIS occurrence records, we have to use the Occurrence core. For this data set, a `parentEvent` is a filtered water sample that was DNA extracted, a sequencing library from that DNA extraction is an `event`, and an `occurrence` is an ASV observed within a library. We will have an an occurence file and a DNA derived data file. Future versions will generate a measurements file.   
**Define files**


### Build a combined Occurrence Core for all Analyses
This code creates an Occurrence Core for each analysis in the submission, then combines them into one Occurence Core. This operation is complex, includes merging dataframes together, adding the missing fields which OBIS and GBIF expect, and parsing taxonomy raw data. 

In [28]:
# --- MAIN DATA PROCESSING LOOP --- Loop through each analysis run defined in params['datafiles']
import pandas as pd
import numpy as np
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

print(f"🚀 Starting data processing for {len(params['datafiles'])} analysis run(s) to generate occurrence records.")

# Define the desired final columns for occurrence.csv IN THE SPECIFIC ORDER REQUIRED
DESIRED_OCCURRENCE_COLUMNS_IN_ORDER = [
    'eventID', 'organismQuantity', 'assay_name', 'occurrenceID', 'verbatimIdentification',
    'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species',  
    'scientificName', 'scientificNameID', 
    'taxonRank', 'identificationRemarks',
    'taxonID', 'basisOfRecord', 'nameAccordingTo', 'organismQuantityType',
    'recordedBy', 'materialSampleID', 'sampleSizeValue', 'sampleSizeUnit',
    'associatedSequences', 'locationID', 'eventDate', 'minimumDepthInMeters', 'maximumDepthInMeters',
    'locality', 'decimalLatitude', 'decimalLongitude',
    'geodeticDatum', 'parentEventID', 'datasetID', 'occurrenceStatus'
]

output_dir = "../processed-v3/" 
os.makedirs(output_dir, exist_ok=True)
output_filename = "occurrence.csv"
output_path = os.path.join(output_dir, output_filename)

all_processed_occurrence_dfs = []
successful_runs = 0
failed_runs = 0

project_recorded_by = "recordedBy_NotProvided"
project_dataset_id = "DatasetID_NotProvided"

def get_project_meta_value(project_meta_df, term_to_find, default_val=pd.NA):
    if not all(col in project_meta_df.columns for col in ['term_name', 'project_level']):
        return default_val
    term_to_find_stripped = str(term_to_find).strip()
    match = project_meta_df[project_meta_df['term_name'].astype(str).str.strip().str.lower() == term_to_find_stripped.lower()]
    if not match.empty:
        value = match['project_level'].iloc[0]
        if pd.notna(value):
            return str(value).strip()
    return default_val

if 'projectMetadata' in data and not data['projectMetadata'].empty:
    project_meta_df = data['projectMetadata']
    project_recorded_by = get_project_meta_value(project_meta_df, 'recordedBy', project_recorded_by)
    project_dataset_id = get_project_meta_value(project_meta_df, 'project_id', project_dataset_id)
else:
    print("  ⚠️ Warning: projectMetadata is empty or not found. 'recordedBy' and 'datasetID' will use default placeholder values.")

# --- MAIN DATA PROCESSING LOOP ---
for analysis_run_name, file_paths_dict in params['datafiles'].items():
    print(f"\nProcessing Analysis Run: {analysis_run_name}")
    try:
        # --- STEP 1: Load and Prepare Raw Taxonomy and Abundance Data ---
        if not (analysis_run_name in raw_data_tables and
                'taxonomy' in raw_data_tables[analysis_run_name] and not raw_data_tables[analysis_run_name]['taxonomy'].empty and
                'occurrence' in raw_data_tables[analysis_run_name] and not raw_data_tables[analysis_run_name]['occurrence'].empty):
            print(f"  Skipping {analysis_run_name}: Raw taxonomy or occurrence data is missing or empty.")
            failed_runs += 1
            continue

        current_tax_df_raw = raw_data_tables[analysis_run_name]['taxonomy'].copy()
        current_abundance_df_raw = raw_data_tables[analysis_run_name]['occurrence'].copy()

        featureid_col_tax = current_tax_df_raw.columns[0]
        current_tax_df_raw.rename(columns={featureid_col_tax: 'featureid'}, inplace=True)
        featureid_col_abun = current_abundance_df_raw.columns[0] 
        current_abundance_df_raw.rename(columns={featureid_col_abun: 'featureid'}, inplace=True)

        sequence_col_dwc = 'DNA_sequence'
        sequence_col_input = 'sequence' 
        confidence_col_original_case = 'Confidence' 
        
        if 'dna_sequence' in current_tax_df_raw.columns and sequence_col_input not in current_tax_df_raw.columns:
             current_tax_df_raw.rename(columns={'dna_sequence': sequence_col_input}, inplace=True)
        elif sequence_col_input not in current_tax_df_raw.columns:
            current_tax_df_raw[sequence_col_input] = pd.NA
        
        # This is the name of the FAIRe column in current_tax_df_raw that holds the string to be used.
        # It will be renamed to 'verbatimIdentification' in current_tax_df_processed.
        source_column_for_verbatim_id = 'taxonomy' 

        if source_column_for_verbatim_id in current_tax_df_raw.columns:
            verbatim_id_source_col = source_column_for_verbatim_id 
            # print(f"  Using column '{verbatim_id_source_col}' from input as source for DwC 'verbatimIdentification'.") # Use for Debug if needed
        else:
            # If the 'taxonomy' column is MISSING from the input file.
            print(f"  CRITICAL WARNING: Column '{source_column_for_verbatim_id}' not found in input taxonomy table for '{analysis_run_name}'. DwC 'verbatimIdentification' will use a placeholder.")
            current_tax_df_raw['verbatimIdentification_placeholder'] = f"Data from '{source_column_for_verbatim_id}' column not available in source"
            verbatim_id_source_col = 'verbatimIdentification_placeholder'
            # This placeholder column will be picked up by tax_cols_to_keep and then renamed.                               
        
        tax_cols_to_keep = ['featureid', sequence_col_input, verbatim_id_source_col]
        if confidence_col_original_case in current_tax_df_raw.columns:
            tax_cols_to_keep.append(confidence_col_original_case)
        else:
            current_tax_df_raw[confidence_col_original_case] = pd.NA 
            tax_cols_to_keep.append(confidence_col_original_case)

        current_tax_df_processed = current_tax_df_raw[[col for col in tax_cols_to_keep if col in current_tax_df_raw.columns]].copy()
        current_tax_df_processed.rename(columns={verbatim_id_source_col: 'verbatimIdentification', 
                                                 sequence_col_input: sequence_col_dwc}, inplace=True)

        # --- STEP 2: Melt Abundance and Merge with Taxonomy ---
        current_assay_occ_melted = pd.melt(
            current_abundance_df_raw, id_vars=['featureid'],
            var_name='samp_name', value_name='organismQuantity'
        )
        current_assay_occ_melted = current_assay_occ_melted[current_assay_occ_melted['organismQuantity'] > 0.0]
        
        current_assay_occurrence_intermediate_df = pd.merge(
            current_assay_occ_melted, current_tax_df_processed,
            on='featureid', how='left'
        )

        # --- STEP 3: Initialize ALL DwC Fields from DESIRED_OCCURRENCE_COLUMNS_IN_ORDER ---
        for col in DESIRED_OCCURRENCE_COLUMNS_IN_ORDER:
            if col not in current_assay_occurrence_intermediate_df.columns:
                 current_assay_occurrence_intermediate_df[col] = pd.NA
        
        # Set values for fields that are constructed or have fixed values for this process
        current_assay_occurrence_intermediate_df['taxonID'] = 'ASV:' + current_assay_occurrence_intermediate_df['featureid'].astype(str)
        current_assay_occurrence_intermediate_df['organismQuantityType'] = 'DNA sequence reads'
        current_assay_occurrence_intermediate_df['occurrenceStatus'] = 'present'
        current_assay_occurrence_intermediate_df['basisOfRecord'] = 'MaterialSample'
        current_assay_occurrence_intermediate_df['nameAccordingTo'] = 'Original Classification; WoRMS/GBIF (pending further matching)'
        
        # --- STEP 4: Assign Project-Level Metadata ---
        current_assay_occurrence_intermediate_df['recordedBy'] = project_recorded_by
        current_assay_occurrence_intermediate_df['datasetID'] = project_dataset_id

        # --- STEP 5: Merge `sampleMetadata` and map FAIRe terms to DwC terms ---
        if 'sampleMetadata' in data and not data['sampleMetadata'].empty:
            sm_df_to_merge = data['sampleMetadata'].copy()
            sm_df_to_merge['samp_name'] = sm_df_to_merge['samp_name'].astype(str).str.strip()
            current_assay_occurrence_intermediate_df['samp_name'] = current_assay_occurrence_intermediate_df['samp_name'].astype(str).str.strip()

            current_assay_occurrence_intermediate_df = pd.merge(
                current_assay_occurrence_intermediate_df, sm_df_to_merge,
                on='samp_name', how='left', suffixes=('', '_sm') 
            )
            
            # These columns have specific assignment logic/calculation later or are core IDs
            # They are filled by mapping only if currently NA. They all have different, specific logic to construct.
            cols_with_specific_logic_or_origin = [
                'datasetID', 'recordedBy', 'eventID', 'occurrenceID', 'taxonID', 
                'organismQuantityType', 'occurrenceStatus', 'basisOfRecord', 'nameAccordingTo',
                'parentEventID', 'associatedSequences', 
                'sampleSizeValue', 'sampleSizeUnit', 'identificationRemarks'
            ]

            # Populate DwC columns using the dwc_data['occurrence'] mapping
            for dwc_col_target, faire_row in dwc_data['occurrence'].iterrows():
                if dwc_col_target not in DESIRED_OCCURRENCE_COLUMNS_IN_ORDER: # Ensure we only care about desired output columns
                    continue 

                faire_col_source_original = str(faire_row['FAIRe_term']).strip()
                source_col_in_df = None
                
                # Check for the column with _sm suffix first (if a clash occurred during merge with sampleMetadata)
                if faire_col_source_original + '_sm' in current_assay_occurrence_intermediate_df.columns:
                    source_col_in_df = faire_col_source_original + '_sm'
                # Else, check for the original FAIRe term name (if no clash)
                elif faire_col_source_original in current_assay_occurrence_intermediate_df.columns:
                    source_col_in_df = faire_col_source_original
                
                if source_col_in_df:
                    # If the DwC target column has specific logic for its creation or is a core ID,
                    # only fill it from sampleMetadata if it's currently NA.
                    if dwc_col_target in cols_with_specific_logic_or_origin:
                        current_assay_occurrence_intermediate_df[dwc_col_target] = current_assay_occurrence_intermediate_df[dwc_col_target].fillna(current_assay_occurrence_intermediate_df[source_col_in_df])
                    else: 
                        # For other "standard" DwC columns (like locality, lat, lon, geodeticDatum, etc.),
                        # directly assign from the source FAIRe column.
                        current_assay_occurrence_intermediate_df[dwc_col_target] = current_assay_occurrence_intermediate_df[source_col_in_df]
                elif dwc_col_target in ['locality', 'decimalLatitude', 'decimalLongitude', 'geodeticDatum', 'eventDate']: # Only print diagnostic for key terms if mapping is missing in checklist
                     print(f"  DIAGNOSTIC: For DwC term '{dwc_col_target}', its mapped FAIRe term '{faire_col_source_original}' (from checklist) was NOT found as a column in the merged sample data (checked as '{faire_col_source_original}' and '{faire_col_source_original}_sm'). The DwC column '{dwc_col_target}' will likely be empty if not populated by other means.")
        else:
            print(f"  Warning: 'sampleMetadata' is empty or not found. Cannot merge for DwC term population for run {analysis_run_name}.")

        # Construct 'locationID' 
        line_id_col_sm = 'line_id_sm' if 'line_id_sm' in current_assay_occurrence_intermediate_df.columns else 'line_id'
        station_id_col_sm = 'station_id_sm' if 'station_id_sm' in current_assay_occurrence_intermediate_df.columns else 'station_id'

        if line_id_col_sm in current_assay_occurrence_intermediate_df.columns and \
           station_id_col_sm in current_assay_occurrence_intermediate_df.columns:
            line_ids = current_assay_occurrence_intermediate_df[line_id_col_sm].astype(str).fillna('NoLineID')
            station_ids = current_assay_occurrence_intermediate_df[station_id_col_sm].astype(str).fillna('NoStationID')
            current_assay_occurrence_intermediate_df['locationID'] = line_ids + "_" + station_ids
            # print(f"    Constructed 'locationID'.") # Reduced verbosity
        else:
            print(f"    Warning: Could not construct 'locationID' using '{line_id_col_sm}' or '{station_id_col_sm}'.")
            if 'locationID' not in current_assay_occurrence_intermediate_df.columns or current_assay_occurrence_intermediate_df['locationID'].isna().all():
                 current_assay_occurrence_intermediate_df['locationID'] = "LocationID_NotAvailable"


        # --- STEP 6: Merge `experimentRunMetadata` & Define `eventID`, `associatedSequences` ---
        assay_name_for_current_run = next((an_key for an_key, runs_dict_val in data.get('analysis_data_by_assay', {}).items() if isinstance(runs_dict_val, dict) and analysis_run_name in runs_dict_val), None)
        
        current_assay_occurrence_intermediate_df['assay_name'] = assay_name_for_current_run # <-- ADD THIS LINE

        if not assay_name_for_current_run:
            print(f"    ERROR: Could not determine assay_name for '{analysis_run_name}'.")
            current_assay_occurrence_intermediate_df['eventID'] = current_assay_occurrence_intermediate_df['eventID'].fillna(f"ERROR_eventID_for_{analysis_run_name}")
            # Also ensure a placeholder for assay_name if it couldn't be found, though it should be an error condition
            current_assay_occurrence_intermediate_df['assay_name'] = current_assay_occurrence_intermediate_df['assay_name'].fillna(f"UNKNOWN_ASSAY_FOR_{analysis_run_name}")
        elif 'experimentRunMetadata' in data and not data['experimentRunMetadata'].empty:
            erm_df = data['experimentRunMetadata'].copy()
            erm_df['samp_name'] = erm_df['samp_name'].astype(str).str.strip()
            erm_df_assay_specific = erm_df[erm_df['assay_name'].astype(str).str.strip() == str(assay_name_for_current_run).strip()]

            if not erm_df_assay_specific.empty:
                faire_lib_id_col = str(dwc_data['occurrence'].loc['eventID', 'FAIRe_term']).strip() if 'eventID' in dwc_data['occurrence'].index else 'lib_id'
                faire_assoc_seq_col = str(dwc_data['occurrence'].loc['associatedSequences', 'FAIRe_term']).strip() if 'associatedSequences' in dwc_data['occurrence'].index else 'associatedSequences'

                cols_to_select_from_erm = {'samp_name'}
                if faire_lib_id_col in erm_df_assay_specific.columns: cols_to_select_from_erm.add(faire_lib_id_col)
                if faire_assoc_seq_col in erm_df_assay_specific.columns: cols_to_select_from_erm.add(faire_assoc_seq_col)
                
                erm_to_merge = erm_df_assay_specific[list(cols_to_select_from_erm)].drop_duplicates(subset=['samp_name']).copy()
                
                current_assay_occurrence_intermediate_df = pd.merge(
                    current_assay_occurrence_intermediate_df, erm_to_merge,
                    on='samp_name', how='left', suffixes=('', '_erm')
                )
                
                source_lib_id_col_actual = faire_lib_id_col + '_erm' if faire_lib_id_col + '_erm' in current_assay_occurrence_intermediate_df.columns else faire_lib_id_col
                if source_lib_id_col_actual in current_assay_occurrence_intermediate_df.columns:
                    current_assay_occurrence_intermediate_df['eventID'] = current_assay_occurrence_intermediate_df['eventID'].fillna(current_assay_occurrence_intermediate_df[source_lib_id_col_actual])
                
                source_assoc_seq_col_actual = faire_assoc_seq_col + '_erm' if faire_assoc_seq_col + '_erm' in current_assay_occurrence_intermediate_df.columns else faire_assoc_seq_col
                if source_assoc_seq_col_actual in current_assay_occurrence_intermediate_df.columns:
                     current_assay_occurrence_intermediate_df['associatedSequences'] = current_assay_occurrence_intermediate_df['associatedSequences'].fillna(current_assay_occurrence_intermediate_df[source_assoc_seq_col_actual])
            else:
                current_assay_occurrence_intermediate_df['eventID'] = current_assay_occurrence_intermediate_df['eventID'].fillna(f"NoExpMeta_eventID_for_{analysis_run_name}")
        
        current_assay_occurrence_intermediate_df['eventID'] = current_assay_occurrence_intermediate_df['eventID'].astype(str)
        
        # --- STEP 7: Construct `occurrenceID` ---
        current_assay_occurrence_intermediate_df['occurrenceID'] = current_assay_occurrence_intermediate_df['eventID'] + '_occ_' + current_assay_occurrence_intermediate_df['featureid'].astype(str)

        # --- STEP 8: `identificationRemarks`, `sampleSizeValue`/`Unit`, `parentEventID` ---
        otu_seq_comp_appr_str = "Unknown sequence comparison approach"
        taxa_class_method_str = ""
        taxa_ref_db_str = "Unknown reference DB"

        if assay_name_for_current_run and analysis_run_name in data.get('analysis_data_by_assay', {}).get(assay_name_for_current_run, {}):
            analysis_meta_df_for_run = data['analysis_data_by_assay'][assay_name_for_current_run][analysis_run_name]
            if 'term_name' in analysis_meta_df_for_run.columns and 'values' in analysis_meta_df_for_run.columns:
                def get_analysis_meta(term, df, default):
                    val_series = df[df['term_name'].astype(str).str.strip() == term]['values']
                    return str(val_series.iloc[0]).strip() if not val_series.empty and pd.notna(val_series.iloc[0]) else default
                otu_seq_comp_appr_str = get_analysis_meta('otu_seq_comp_appr', analysis_meta_df_for_run, otu_seq_comp_appr_str)
                taxa_class_method_str = get_analysis_meta('taxa_class_method', analysis_meta_df_for_run, taxa_class_method_str)
                taxa_ref_db_str = get_analysis_meta('otu_db', analysis_meta_df_for_run, taxa_ref_db_str)
        
        confidence_value_series = pd.Series(["unknown confidence"] * len(current_assay_occurrence_intermediate_df), index=current_assay_occurrence_intermediate_df.index, dtype=object)
        if confidence_col_original_case in current_assay_occurrence_intermediate_df.columns:
            confidence_value_series = current_assay_occurrence_intermediate_df[confidence_col_original_case].astype(str).fillna("unknown confidence")
        
        current_assay_occurrence_intermediate_df['identificationRemarks'] = f"{otu_seq_comp_appr_str}, confidence: " + confidence_value_series + f", against reference database: {taxa_ref_db_str}"
        
        if 'eventID' in current_assay_occurrence_intermediate_df.columns and not current_assay_occurrence_intermediate_df['eventID'].isna().all():
            sample_size_map = current_assay_occurrence_intermediate_df.groupby('eventID')['organismQuantity'].sum().to_dict()
            current_assay_occurrence_intermediate_df['sampleSizeValue'] = current_assay_occurrence_intermediate_df['eventID'].map(sample_size_map)
        current_assay_occurrence_intermediate_df['sampleSizeUnit'] = 'DNA sequence reads'
        
        if 'parentEventID' in dwc_data['occurrence'].index:
            faire_parent_event_id_col_name = str(dwc_data['occurrence'].loc['parentEventID','FAIRe_term']).strip() 
            actual_parent_event_col_name = faire_parent_event_id_col_name + "_sm" if faire_parent_event_id_col_name + "_sm" in current_assay_occurrence_intermediate_df.columns else faire_parent_event_id_col_name
            if actual_parent_event_col_name in current_assay_occurrence_intermediate_df.columns :
                 current_assay_occurrence_intermediate_df['parentEventID'] = current_assay_occurrence_intermediate_df['parentEventID'].fillna(current_assay_occurrence_intermediate_df[actual_parent_event_col_name])

        # --- STEP 9: Final Column Selection and Order for this assay's DataFrame ---
        for col_final_desired in DESIRED_OCCURRENCE_COLUMNS_IN_ORDER:
            if col_final_desired not in current_assay_occurrence_intermediate_df.columns:
                current_assay_occurrence_intermediate_df[col_final_desired] = pd.NA
        
        current_assay_occurrence_final_df = current_assay_occurrence_intermediate_df[DESIRED_OCCURRENCE_COLUMNS_IN_ORDER].copy()
        
        all_processed_occurrence_dfs.append(current_assay_occurrence_final_df)
        successful_runs += 1
        print(f"  Successfully processed {analysis_run_name}: Generated {len(current_assay_occurrence_final_df)} records.")

    except Exception as e:
        import traceback
        print(f"  ❌ Error processing {analysis_run_name}: {str(e)}")
        print(f"  Traceback for {analysis_run_name}: {traceback.format_exc()}")
        failed_runs += 1

# --- POST-LOOP CONCATENATION & FINALIZATION ---
print(f"\n🏁 LOOP COMPLETED: Successful runs: {successful_runs}, Failed runs: {failed_runs}, Total DataFrames to combine: {len(all_processed_occurrence_dfs)}")

if all_processed_occurrence_dfs:
    occ_all_final_combined = pd.concat(all_processed_occurrence_dfs, ignore_index=True, sort=False)
    
    for col_final_desired in DESIRED_OCCURRENCE_COLUMNS_IN_ORDER:
        if col_final_desired not in occ_all_final_combined.columns:
            occ_all_final_combined[col_final_desired] = pd.NA
    
    occ_all_final_output = occ_all_final_combined.reindex(columns=DESIRED_OCCURRENCE_COLUMNS_IN_ORDER)
    
    original_rows_before_dedup = len(occ_all_final_output)
    if 'occurrenceID' in occ_all_final_output.columns and not occ_all_final_output['occurrenceID'].isna().all():
        num_duplicates = occ_all_final_output.duplicated(subset=['occurrenceID']).sum()
        if num_duplicates > 0:
            occ_all_final_output.drop_duplicates(subset=['occurrenceID'], keep='first', inplace=True)
            print(f"🔄 Dropped {num_duplicates} duplicate occurrenceID records. Final rows: {len(occ_all_final_output)}.")
        else:
            print("🔄 No duplicate occurrenceID records found to drop.")
    else:
        print("  ⚠️ WARNING: 'occurrenceID' column not found or is all NA. Cannot effectively drop duplicates based on it.")

    try:
        occ_all_final_output.to_csv(output_path, index=False, na_rep='') 
        print(f"\n💾 Combined occurrence file '{output_filename}' saved to '{output_path}' with {len(occ_all_final_output)} records.")
        print(f"\n👀 Preview of final combined occurrence data (first 5 rows, selected columns):")
        preview_cols_subset = ['eventID', 'occurrenceID', 'assay_name', 'parentEventID', 'datasetID', 'recordedBy', 
                               'locality', 'decimalLatitude', 'decimalLongitude', 'geodeticDatum', 
                               'identificationRemarks', 'locationID']
        preview_cols_to_show = [col for col in preview_cols_subset if col in occ_all_final_output.columns]
        display(occ_all_final_output[preview_cols_to_show].head())
    except Exception as e:
        print(f"  ❌ Error saving combined occurrence file: {str(e)}")
else:
    print(f"❌ No data to combine - all analysis runs may have failed or yielded no occurrence records.")


🚀 Starting data processing for 3 analysis run(s) to generate occurrence records.

Processing Analysis Run: gomecc4_18s_p1-6_v2024.10_241122
  Successfully processed gomecc4_18s_p1-6_v2024.10_241122: Generated 147083 records.

Processing Analysis Run: gomecc4_16s_p3-6_v2024.10_241122
  Successfully processed gomecc4_16s_p3-6_v2024.10_241122: Generated 118671 records.

Processing Analysis Run: gomecc4_16s_p1-2_v2024.10_241122
  Successfully processed gomecc4_16s_p1-2_v2024.10_241122: Generated 46460 records.

🏁 LOOP COMPLETED: Successful runs: 3, Failed runs: 0, Total DataFrames to combine: 3
🔄 No duplicate occurrenceID records found to drop.

💾 Combined occurrence file 'occurrence.csv' saved to '../processed-v3/occurrence.csv' with 312214 records.

👀 Preview of final combined occurrence data (first 5 rows, selected columns):


,eventID,occurrenceID,assay_name,parentEventID,datasetID,recordedBy,locality,decimalLatitude,decimalLongitude,geodeticDatum,identificationRemarks,locationID
0,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_36aa75f9b28f5f831c2d631ba65c2bcb,ssu18sv9-emp,GOMECC4_27N_Sta1_DCM_A,noaa-aoml-gomecc4,Luke Thompson,"USA: Atlantic Ocean, east of Florida (27 N)",26.997,-79.618,WGS84,"qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1, confidence: 0.99999999990944, against reference database: PR2 v5.0.1; V9 1391f-1510r r...",27N_Sta1
1,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_4e38e8ced9070952b314e1880bede1ca,ssu18sv9-emp,GOMECC4_27N_Sta1_DCM_A,noaa-aoml-gomecc4,Luke Thompson,"USA: Atlantic Ocean, east of Florida (27 N)",26.997,-79.618,WGS84,"qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1, confidence: 0.999067062720315, against reference database: PR2 v5.0.1; V9 1391f-1510r ...",27N_Sta1
2,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_2a31e5c01634165da99e7381279baa75,ssu18sv9-emp,GOMECC4_27N_Sta1_DCM_A,noaa-aoml-gomecc4,Luke Thompson,"USA: Atlantic Ocean, east of Florida (27 N)",26.997,-79.618,WGS84,"qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1, confidence: 0.8911679667827849, against reference database: PR2 v5.0.1; V9 1391f-1510r...",27N_Sta1
3,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_ecee60339b2fb88ea6d1c8d18054bed4,ssu18sv9-emp,GOMECC4_27N_Sta1_DCM_A,noaa-aoml-gomecc4,Luke Thompson,"USA: Atlantic Ocean, east of Florida (27 N)",26.997,-79.618,WGS84,"qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1, confidence: 0.9996383713806553, against reference database: PR2 v5.0.1; V9 1391f-1510r...",27N_Sta1
4,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_fa1f1a97dd4ae7c826009186bad26384,ssu18sv9-emp,GOMECC4_27N_Sta1_DCM_A,noaa-aoml-gomecc4,Luke Thompson,"USA: Atlantic Ocean, east of Florida (27 N)",26.997,-79.618,WGS84,"qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1, confidence: 0.97066987594629, against reference database: PR2 v5.0.1; V9 1391f-1510r r...",27N_Sta1


## NOTE:
The Occurrence Core at this step (before taxonomic assignment through WoRMS or GBIF), contains an assay_name column. This will be removed from the final Occurrence Core (after taxonomic assignment) but it is used by the taxonomic assignment code to know which assay's data you want to remove the 'species' rank from consideration. This is because some assays, like 16S for example, return non-usable assignments at species level, while, for example, 18S species assignments ARE useful.

# DEBUGGING ONLY: Create Smaller Subset for Faster Taxonomic Matching

**Run this cell ONLY if you want to test taxonomic matching on a small subset.**
This cell will take the `occ_df_for_processing_step` (which currently holds the full dataset from Cell 33), create a small subset from it, and then **overwrite** `occ_df_for_processing_step` with this smaller subset.
If you want to run taxonomic matching on the **full dataset, simply DO NOT RUN THIS CELL.**

In [28]:
# Create a small subset of occurrence.csv for faster testing of taxonomic matching

import pandas as pd
import os

# Determine the correct output directory path
current_output_dir = None
if 'output_dir' in globals() and isinstance(output_dir, str):
    current_output_dir = output_dir # From a previous run of cell 28
elif 'params' in globals() and 'output_dir' in params and isinstance(params['output_dir'], str):
    current_output_dir = params['output_dir'] # From cell 8
else:
    current_output_dir = "../processed-v3/" # Fallback
    print(f"Warning: 'output_dir' not found in globals or params, using default: {current_output_dir}")

full_occurrence_csv_path = os.path.join(current_output_dir, "occurrence.csv")
occ_df_subset_for_matching = pd.DataFrame() # Initialize
full_occ_df = pd.DataFrame() # Initialize

N_ROWS_PER_ASSAY_SUBSET = 10 # Number of rows to take per assay for the subset
print(f"Attempting to load full occurrence data from: {full_occurrence_csv_path}")
if os.path.exists(full_occurrence_csv_path):
    try:
        full_occ_df = pd.read_csv(full_occurrence_csv_path, low_memory=False, dtype=str)
        if 'organismQuantity' in full_occ_df.columns:
            full_occ_df['organismQuantity'] = pd.to_numeric(full_occ_df['organismQuantity'], errors='coerce').fillna(0).astype(float)
        print(f"Successfully loaded {full_occurrence_csv_path} with {len(full_occ_df)} records.")

        if not full_occ_df.empty and 'assay_name' in full_occ_df.columns and 'verbatimIdentification' in full_occ_df.columns:
            available_assays = full_occ_df['assay_name'].dropna().unique()
            print(f"  Available assays in occurrence.csv: {available_assays}")
            
            subset_dfs = []
            for assay in available_assays:
                assay_subset = full_occ_df[full_occ_df['assay_name'] == assay].head(N_ROWS_PER_ASSAY_SUBSET)
                if not assay_subset.empty:
                    subset_dfs.append(assay_subset)
                    print(f"  Selected {len(assay_subset)} rows for assay '{assay}' in the subset.")
                else:
                    print(f"  No rows found for assay '{assay}' to include in subset.")

            if subset_dfs:
                occ_df_subset_for_matching = pd.concat(subset_dfs, ignore_index=True)
                print(f"Successfully created subset with {len(occ_df_subset_for_matching)} total rows for taxonomic matching.")
                print(f"  Unique assay_names in subset: {occ_df_subset_for_matching['assay_name'].unique()}")
                # For inspection:
                # display(occ_df_subset_for_matching[['eventID', 'occurrenceID', 'verbatimIdentification', 'assay_name']].head())
            else:
                print("Could not create a valid subset. No assay data found or all assays resulted in empty subsets.")
        else:
            print("Full occurrence DataFrame is empty or missing 'assay_name'/'verbatimIdentification' columns. Cannot create subset.")

    except Exception as e:
        print(f"Error loading or processing {full_occurrence_csv_path} for subsetting: {e}")
        import traceback
        print(traceback.format_exc())
else:
    print(f"ERROR: Full occurrence file not found at {full_occurrence_csv_path}. Cannot create subset. Ensure Cell 28 (occurrence.csv generation) ran successfully.")

# Determine which DataFrame to use for the actual matching process
current_run_type = "NO DATA"
if not occ_df_subset_for_matching.empty:
    occ_df_for_actual_matching = occ_df_subset_for_matching
    current_run_type = "SUBSET"
    print(f"\nProceeding with the {current_run_type} ({len(occ_df_for_actual_matching)} rows) for taxonomic matching.")
elif not full_occ_df.empty: # Fallback to full if subset is empty but full loaded
    print("\nWARNING: Subset creation failed or resulted in an empty DataFrame. Proceeding with FULL dataset.")
    occ_df_for_actual_matching = full_occ_df
    current_run_type = "FULL DATASET"
else:
    print("\nERROR: Neither subset nor full occurrence data is available for matching. Please check previous cells.")
    occ_df_for_actual_matching = pd.DataFrame() # Ensure it's an empty DF

Attempting to load full occurrence data from: ../processed-v3/occurrence.csv
Successfully loaded ../processed-v3/occurrence.csv with 312214 records.
  Available assays in occurrence.csv: ['ssu18sv9-emp' 'ssu16sv4v5-emp']
  Selected 10 rows for assay 'ssu18sv9-emp' in the subset.
  Selected 10 rows for assay 'ssu16sv4v5-emp' in the subset.
Successfully created subset with 20 total rows for taxonomic matching.
  Unique assay_names in subset: ['ssu18sv9-emp' 'ssu16sv4v5-emp']

Proceeding with the SUBSET (20 rows) for taxonomic matching.


## Taxonomic Assignment

The following cells will perform taxonomic matching using the API specified in the notebook parameters (`params['taxonomic_api_source']`). This process will:
1. Dynamically import the appropriate matching script (e.g., `WoRMS_v3_matching.py` or a future `GBIF_v3_matching.py`).
2. Read the `occurrence.csv` file generated previously (which includes raw `verbatimIdentification` strings and `assay_name`).
3. Use the imported script to query the selected API.
4. Implement caching to avoid redundant API calls.
5. Handle assay-specific rules (e.g., skipping species-level matching for specified assays).
6. Update the occurrence data with API-derived `scientificName`, `scientificNameID`, `taxonRank`, and hierarchical rank columns.
7. Set `nameAccordingTo` to reflect the API source.
8. Handle specific post-processing rules, such as for "Eukaryota"-only verbatim strings.

In [29]:
import importlib
import pandas as pd
import os
import WoRMS_v3_matching # Import the new script

# Reload the module to pick up any changes if you edit the .py file
importlib.reload(WoRMS_v3_matching)

print("Successfully imported and reloaded WoRMS_v3_matching.py")

# --- Define Parameters for Taxonomic Matching ---

# This should be defined in your main parameters cell (e.g., Cell 8 of your notebook)
# Ensure 'params' dictionary exists from that cell.
if 'params' not in globals():
    print("CRITICAL ERROR: 'params' dictionary not found. It should be defined in an early cell (e.g., Cell 8).")
    params = {} # Initialize to prevent immediate crash, but this is not ideal

# 1. API Source (already in your params from Cell 8)
# params['taxonomic_api_source'] = 'WoRMS' # or 'GBIF' when ready

# 2. Assays to skip SPECIES-level matching for (already in your params from Cell 8)
# params['user_defined_assays_to_skip_species'] = ['ssu16sv4v5-emp'] # Example

# 3. Number of processes for matching (0 means use all available CPUs)
# This can also be in your main params cell or set here.
params['worms_n_proc'] = params.get('worms_n_proc', 0) 
params['gbif_n_proc'] = params.get('gbif_n_proc', 0) # For future GBIF use

# 4. Output directory (should be consistent with where occurrence.csv was saved)
# This should also ideally come from your main params cell (Cell 8) or the occurrence.csv generation cell (Cell 28)
if 'output_dir' not in params:
    if 'output_dir' in globals() and isinstance(output_dir, str): # If set by cell 28
         params['output_dir'] = output_dir
    else:
        params['output_dir'] = "../processed-v3/" # Fallback
        print(f"Warning: 'output_dir' not found in params, using fallback: {params['output_dir']}")
else: # If it is in params, ensure it's a string
    if not isinstance(params['output_dir'], str):
        params['output_dir'] = "../processed-v3/"
        print(f"Warning: params['output_dir'] was not a string, reset to default: {params['output_dir']}")


# --- Consolidate parameters for the matching function ---
# The WoRMS_OBIS_matcher.py script will primarily use:
# - params_dict['taxonomic_api_source']
# - params_dict['assays_to_skip_species_match'] (derived from user_defined_assays_to_skip_species)
# - params_dict['worms_n_proc'] (or 'gbif_n_proc' for GBIF)

current_api_source = params.get('taxonomic_api_source', 'WoRMS') # Default to WoRMS if not set
print(f"\nTaxonomic Matching Setup:")
print(f"  API Source: {current_api_source}")
print(f"  Assays configured by user to skip species-level matching: {params.get('user_defined_assays_to_skip_species', 'NONE DEFINED')}")
if current_api_source == 'WoRMS':
    print(f"  Number of processes for WoRMS: {params['worms_n_proc'] if params['worms_n_proc'] > 0 else 'All available CPUs'}")
elif current_api_source == 'GBIF':
    print(f"  Number of processes for GBIF: {params['gbif_n_proc'] if params['gbif_n_proc'] > 0 else 'All available CPUs'} (GBIF matching not yet fully implemented)")
print(f"  Output directory for reading/writing files: {params['output_dir']}")

# Standard DwC ranks that the matching script will try to populate
DWC_RANKS_STD = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

Successfully imported and reloaded WoRMS_v3_matching.py

Taxonomic Matching Setup:
  API Source: WoRMS
  Assays configured by user to skip species-level matching: ['ssu16sv4v5-emp']
  Number of processes for WoRMS: All available CPUs
  Output directory for reading/writing files: ../processed-v3/


### OPTIONAL: Use local reference database for 18S taxonomies
This step is entirely optional. You can either continue and run this cell to SIGNIFICANTLY speed up taxonomic assignment for 18S taxonomies, or skip it, and edna2obis will still function.

In [30]:
# OPTIONAL: Pre-computation using PR2 database for 18S AphiaID mapping
#
# This cell loads the PR2 reference database, which contains pre-mapped WoRMS AphiaIDs for many 18S taxa.
# By running this, you create a dictionary that the main matching function can use to bypass the slow,
# name-based API search for these specific taxa, significantly speeding up the process for 18S data.
#
# If you do not have the PR2 file or wish to skip this step, simply DO NOT run this cell.
# The main script will proceed without this optimization.

import pandas as pd
import numpy as np
import os

# --- Parameters for this step ---
# Define the path to your local copy of the PR2 database Excel file.
# You may need to download it first and place it in a relevant directory.
pr2_database_path = "../raw-v3/pr2_version_5.0.0_taxonomy.xlsx" # <-- USER: UPDATE THIS PATH IF NEEDED

# --- Logic to create the AphiaID dictionary ---
pr2_worms_dict = {} # Initialize an empty dictionary

if os.path.exists(pr2_database_path):
    print(f"Loading PR2 database from: {pr2_database_path}")
    try:
        pr2_df = pd.read_excel(pr2_database_path, index_col=None, na_values=[""])
        
        # Filter for rows that have a WoRMS ID and a species name
        pr2_df.dropna(subset=['worms_id', 'species'], inplace=True)
        
        # Clean up the species names to match the format in our main data
        # (e.g., replacing underscores, removing 'sp.', etc.)
        species_cleaned = pr2_df['species'].str.replace('_', ' ', regex=False)
        species_cleaned = species_cleaned.str.replace(' sp.', '', regex=False).str.strip()
        
        # Create the dictionary: {species_name: aphia_id}
        pr2_worms_dict = dict(zip(species_cleaned, pr2_df['worms_id'].astype(int)))
        
        print(f"Successfully created PR2-WoRMS AphiaID dictionary with {len(pr2_worms_dict)} entries.")

    except Exception as e:
        print(f"---")
        print(f"WARNING: Could not load or process the PR2 database file at the specified path.")
        print(f"Error: {e}")
        print("The taxonomic matching will proceed without AphiaID pre-matching.")
        print(f"---")
        pr2_worms_dict = {} 
else:
    print(f"---")
    print(f"WARNING: PR2 database file not found at: {pr2_database_path}")
    print("The taxonomic matching will proceed without AphiaID pre-matching.")
    print(f"---")

# Make the dictionary available to the matching script by adding it to the main params
if 'params' in locals():
    params['pr2_worms_dict'] = pr2_worms_dict

Loading PR2 database from: ../raw-v3/pr2_version_5.0.0_taxonomy.xlsx
Successfully created PR2-WoRMS AphiaID dictionary with 13553 entries.


Now we will determine which ranks each assay uses in this submission, based on the raw taxonomy data files. This is essential, it enables you to specify which assays you would like to NOT have species be considered during taxonomic assignment. (Sometimes, certain assays 'species' ranks are not useful, and can make results inaccurate).

In [31]:
# --- STEP A: Determine Maximum Ranks per Assay (User's Corrected Logic) ---
# This cell implements the user's correct logic to determine the number of taxonomic ranks
# for each assay by inspecting the structure of the asvTaxaFeatures files.

print("📝 Determining maximum taxonomic ranks for each assay...")
assay_rank_info = {}
for analysis_run_name, tables in raw_data_tables.items():
    tax_df = tables['taxonomy']
    assay_name = next((assay for assay, runs in data['analysis_data_by_assay'].items() if analysis_run_name in runs), None)
    if assay_name:
        try:
            # Find columns between the two guaranteed markers
            start_idx = tax_df.columns.get_loc('verbatimIdentification') + 1
            end_idx = tax_df.columns.get_loc('Confidence')
            rank_cols = tax_df.columns[start_idx:end_idx].tolist()
            
            assay_rank_info[assay_name] = {'max_depth': len(rank_cols)}
            print(f"  - Assay '{assay_name}': Found {len(rank_cols)} rank columns.")
        except KeyError:
            print(f"  ⚠️ WARNING: Could not find required columns for assay '{assay_name}'. Using a default of 7 ranks.")
            assay_rank_info[assay_name] = {'max_depth': 7}

# Pass this information to the main params dictionary for the next step
params['assay_rank_info'] = assay_rank_info
print("✅ Rank information stored successfully.")

📝 Determining maximum taxonomic ranks for each assay...
  - Assay 'ssu18sv9-emp': Found 9 rank columns.
  - Assay 'ssu16sv4v5-emp': Found 7 rank columns.
  - Assay 'ssu16sv4v5-emp': Found 7 rank columns.
✅ Rank information stored successfully.


### Perform Taxonomic Matching

This cell loads the intermediate `occurrence.csv` file (which should include the `assay_name` column) and applies the taxonomic matching using the selected API and its parameters. DEBUG statements will show progress.

NOTE: This operation may take a few minutes

In [32]:
# --- Perform Taxonomic Matching ---
# This cell calls the matching script which handles the API queries and 
# returns the full, taxonomically-enriched DataFrame, following the proven-working pattern.

# This variable comes from the successful run of the cell that creates the combined occurrence core
df_to_match = occ_all_final_output.copy() 
print(f"INFO: Proceeding with the full dataset ({len(df_to_match)} rows) for taxonomic matching.")
current_run_type = "FULL DATASET" # Set for consistent output messaging

# Proceed with matching if the DataFrame is not empty
if df_to_match.empty:
    print("Cannot proceed with matching as the dataframe is empty.")
    occ_df_matched_with_api = pd.DataFrame()
elif 'params' not in globals():
    print("ERROR: 'params' dictionary not found. Cannot proceed with taxonomic matching.")
    occ_df_matched_with_api = pd.DataFrame()
else:
    api_to_use = params.get('taxonomic_api_source')
    print(f"\nStarting taxonomic matching using '{api_to_use}' API on {current_run_type} data...")

    occ_df_matched_with_api = pd.DataFrame()
    if api_to_use == 'WoRMS':
        # Consolidate all parameters needed by the matching script.
        # This combines the structure from the user's working example with the new
        # parameters needed for PR2/assay-specific logic.
        matching_params = {
            'taxonomic_api_source': 'WoRMS',
            'assays_to_skip_species_match': params.get('user_defined_assays_to_skip_species', []),
            'pr2_worms_dict': params.get('pr2_worms_dict', {}),
            'assay_rank_info': params.get('assay_rank_info', {})
        }
        
        # Call the main function from the helper script.
        # This function is expected to return the *entire* dataframe with taxonomic columns already added/updated.
        # This avoids the large memory merge that caused the error previously.
        occ_df_matched_with_api = WoRMS_v3_matching.get_worms_match_for_dataframe(
            occurrence_df=df_to_match,
            params_dict=matching_params,
            n_proc=params.get('worms_n_proc', 0)
        )
    else:
        print(f"API '{api_to_use}' is not yet implemented.")
        occ_df_matched_with_api = df_to_match.copy()

    print(f"\nTaxonomic matching process via '{api_to_use}' finished.")

    # --- Display Preview ---
    if not occ_df_matched_with_api.empty:
        print("\nPreview of DataFrame after matching (selected columns):")
        preview_cols = ['eventID', 'occurrenceID', 'verbatimIdentification', 'assay_name', 
                        'scientificName', 'scientificNameID', 'taxonRank', 'nameAccordingTo', 'match_type_debug'] + DWC_RANKS_STD
        display_cols = [col for col in preview_cols if col in occ_df_matched_with_api.columns]
        display(occ_df_matched_with_api[display_cols].head())
        
        if 'match_type_debug' in occ_df_matched_with_api.columns:
            print(f"\nCounts of '{api_to_use}' match_type_debug:")
            print(occ_df_matched_with_api['match_type_debug'].value_counts(dropna=False))
        if 'scientificName' in occ_df_matched_with_api.columns:
            print(f"\nCounts of resulting '{api_to_use}' scientificName (top 10 unique non-NA values):")
            print(occ_df_matched_with_api['scientificName'].dropna().value_counts().head(10))
    else:
        print(f"DataFrame 'occ_df_matched_with_api' is empty after '{api_to_use}' matching attempt.")

2025-06-11 16:46:41,161 - INFO - Found 3908 unique combinations to process.
2025-06-11 16:46:41,162 - INFO - Starting Stage 1: AphiaID pre-matching.


INFO: Proceeding with the full dataset (312214 rows) for taxonomic matching.

Starting taxonomic matching using 'WoRMS' API on FULL DATASET data...


2025-06-11 16:50:49,588 - INFO - Finished Stage 1. Matched 339 taxa via AphiaID. Remaining: 3569.
2025-06-11 16:50:49,604 - INFO - Starting Stage 2: Batch name matching for 3768 unique terms.
2025-06-11 17:00:52,562 - INFO - Finished Stage 2. Remaining unmatched: 42.
2025-06-11 17:00:52,564 - INFO - Starting Stage 3: Assigning 'incertae sedis' to 42 remaining taxa.
2025-06-11 17:00:52,564 - INFO - Applying all results to DataFrame.
2025-06-11 17:00:54,495 - INFO - 
--- WoRMS Matching Complete ---
2025-06-11 17:00:54,496 - INFO - Total processing time: 853.50 seconds.



Taxonomic matching process via 'WoRMS' finished.

Preview of DataFrame after matching (selected columns):


,eventID,occurrenceID,verbatimIdentification,assay_name,scientificName,scientificNameID,taxonRank,nameAccordingTo,match_type_debug,kingdom,phylum,class,order,family,genus,species
0,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_36aa75f9b28f5f831c2d631ba65c2bcb,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda,ssu18sv9-emp,Crustacea,urn:lsid:marinespecies.org:taxname:1066,Subphylum,WoRMS,Success_Batch_Crustacea,Animalia,Arthropoda,<NA>,<NA>,<NA>,<NA>,<NA>
1,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_4e38e8ced9070952b314e1880bede1ca,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Clausocalanus;Clausocalanus_furcatus;,ssu18sv9-emp,Clausocalanus furcatus,urn:lsid:marinespecies.org:taxname:104503,Species,WoRMS,Success_AphiaID_104503,Animalia,Arthropoda,Copepoda,Calanoida,Clausocalanidae,Clausocalanus,<NA>
2,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_2a31e5c01634165da99e7381279baa75,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Oithona;Oithona_sp.;,ssu18sv9-emp,Oithona,urn:lsid:marinespecies.org:taxname:106485,Genus,WoRMS,Success_Batch_Oithona sp.,Animalia,Arthropoda,Copepoda,Cyclopoida,Oithonidae,Oithona,<NA>
3,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_ecee60339b2fb88ea6d1c8d18054bed4,Eukaryota;TSAR;Alveolata;Dinoflagellata;Dinophyceae,ssu18sv9-emp,Dinophyceae,urn:lsid:marinespecies.org:taxname:19542,Class,WoRMS,Success_Batch_Dinophyceae,Chromista,Myzozoa,Dinophyceae,<NA>,<NA>,<NA>,<NA>
4,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_fa1f1a97dd4ae7c826009186bad26384,Eukaryota;TSAR;Alveolata;Dinoflagellata;Dinophyceae;Gymnodiniales;Gymnodiniaceae,ssu18sv9-emp,Gymnodiniaceae,urn:lsid:marinespecies.org:taxname:109410,Family,WoRMS,Success_Batch_Gymnodiniaceae,Chromista,Myzozoa,Dinophyceae,Gymnodiniales,Gymnodiniaceae,<NA>,<NA>



Counts of 'WoRMS' match_type_debug:
match_type_debug
Success_Batch_Alphaproteobacteria              38874
Success_Batch_Syndiniales                      34619
Failed_All_Stages_IncertaeSedis                22762
Success_Batch_Gammaproteobacteria              14519
Success_Batch_Marinimicrobia (SAR406 clade)    13342
                                               ...  
Success_Batch_Enterovibrio                         1
Success_AphiaID_134541                             1
Success_Batch_Muricauda                            1
Success_AphiaID_106306                             1
Success_Batch_Haloplasma                           1
Name: count, Length: 1337, dtype: int64

Counts of resulting 'WoRMS' scientificName (top 10 unique non-NA values):
scientificName
Alphaproteobacteria    38874
Syndiniales            34998
incertae sedis         22762
Gammaproteobacteria    14519
Marinimicrobium        13343
Bacteria               13165
Dinophyceae             9867
Rhizaria                7121
C

### Post-Matching Processing and Final Save

This cell handles API-specific post-processing. For example, if WoRMS was used, it checks for verbatim strings that were effectively "Eukaryota"-only and ensures they are set to "incertae sedis" if not already resolved to a lower rank. The temporary `assay_name` and `match_type_debug` columns are then removed, and the file is saved.

In [33]:
# Post-Matching Processing and Final Save

if 'occ_df_matched_with_api' not in globals() or occ_df_matched_with_api.empty:
    print("Skipping post-matching and save: 'occ_df_matched_with_api' is not defined or is empty.")
else:
    print("Starting post-matching processing.")
    api_source_used = params.get('taxonomic_api_source', 'UnknownAPI') # Get from params
    
    # --- Handle "Eukaryota"-only verbatim strings specifically after WoRMS matching ---
    if api_source_used == 'WoRMS':
        eukaryota_verbatim_mask = pd.Series([False] * len(occ_df_matched_with_api), index=occ_df_matched_with_api.index)
        if 'verbatimIdentification' in occ_df_matched_with_api.columns and \
           'scientificName' in occ_df_matched_with_api.columns and \
           'taxonRank' in occ_df_matched_with_api.columns:

            for index, row in occ_df_matched_with_api.iterrows():
                vi = str(row.get('verbatimIdentification', '')).strip().lower()
                parsed_vi = [term.strip() for term in vi.split(';') if term.strip()]
                
                is_effectively_eukaryota_only_verbatim = False
                if len(parsed_vi) == 1 and parsed_vi[0] == 'eukaryota':
                    is_effectively_eukaryota_only_verbatim = True
                elif len(parsed_vi) > 0 and parsed_vi[-1] == 'eukaryota': 
                    preceding_meaningful = [term for term in parsed_vi[:-1] if term and term != 'unassigned']
                    if not preceding_meaningful: # If only "eukaryota" is meaningful at the end
                        is_effectively_eukaryota_only_verbatim = True
                
                if is_effectively_eukaryota_only_verbatim:
                    current_sn = str(row.get('scientificName', '')).lower()
                    current_tr = str(row.get('taxonRank', '')).lower()
                    high_ranks = ['kingdom', 'superkingdom', 'domain', 'subkingdom', 'no rank', 'unranked', ''] # Include empty/None as high rank
                    
                    if current_sn == 'eukaryota' and current_tr in high_ranks:
                        eukaryota_verbatim_mask[index] = True
                    elif current_sn == 'incertae sedis' and is_effectively_eukaryota_only_verbatim:
                         eukaryota_verbatim_mask[index] = True


            num_eukaryota_verbatim_to_convert = eukaryota_verbatim_mask.sum()
            if num_eukaryota_verbatim_to_convert > 0:
                print(f"  Post-processing ({api_source_used}): Found {num_eukaryota_verbatim_to_convert} records with 'Eukaryota'-only verbatim strings that matched to high-level Eukaryota or were already 'incertae sedis'. Standardizing to 'incertae sedis'.")
                occ_df_matched_with_api.loc[eukaryota_verbatim_mask, 'scientificName'] = 'incertae sedis'
                occ_df_matched_with_api.loc[eukaryota_verbatim_mask, 'scientificNameID'] = 'urn:lsid:marinespecies.org:taxname:12' # AphiaID for Incertae Sedis in WoRMS
                occ_df_matched_with_api.loc[eukaryota_verbatim_mask, 'taxonRank'] = 'no rank'
                for rank_col in DWC_RANKS_STD: # Clear out other ranks
                    if rank_col in occ_df_matched_with_api.columns:
                        occ_df_matched_with_api.loc[eukaryota_verbatim_mask, rank_col] = None 
                if 'match_type_debug' in occ_df_matched_with_api.columns:
                     occ_df_matched_with_api.loc[eukaryota_verbatim_mask, 'match_type_debug'] = 'Standardized_IncertaeSedis_From_EukaryotaOnlyVerbatim'
                if 'nameAccordingTo' in occ_df_matched_with_api.columns:
                    occ_df_matched_with_api.loc[eukaryota_verbatim_mask, 'nameAccordingTo'] = api_source_used + "; Local rule for Eukaryota-only verbatim"


    # --- Remove temporary/intermediate columns ---
    columns_to_drop_final = []
    if 'assay_name' in occ_df_matched_with_api.columns: columns_to_drop_final.append('assay_name')
    if 'match_type_debug' in occ_df_matched_with_api.columns: columns_to_drop_final.append('match_type_debug')
    
    if columns_to_drop_final:
        occ_df_matched_with_api.drop(columns=columns_to_drop_final, inplace=True, errors='ignore')
        print(f"  Removed temporary columns: {columns_to_drop_final}")

    # --- Define final column order ---
    if 'DESIRED_OCCURRENCE_COLUMNS_IN_ORDER' in globals():
        final_dwc_columns_ordered_post_match = [col for col in DESIRED_OCCURRENCE_COLUMNS_IN_ORDER if col != 'assay_name']
    else: 
        print("Warning: DESIRED_OCCURRENCE_COLUMNS_IN_ORDER not found. Using columns present in DataFrame.")
        final_dwc_columns_ordered_post_match = occ_df_matched_with_api.columns.tolist()

    # Ensure all desired columns exist, add as NA if missing
    for col in final_dwc_columns_ordered_post_match:
        if col not in occ_df_matched_with_api.columns:
            occ_df_matched_with_api[col] = pd.NA 
            print(f"  Warning: Final column '{col}' was missing from matched data and added as NA.")

    occ_df_final_output = occ_df_matched_with_api.reindex(columns=final_dwc_columns_ordered_post_match)
    
    # --- Save the final, taxonomically enriched occurrence file ---
    final_occurrence_filename = f"occurrence_{api_source_used.lower()}_matched.csv" 
    
    output_directory_for_save = params.get('output_dir', "../processed-v3/")
    if not isinstance(output_directory_for_save, str): # Safeguard
        output_directory_for_save = "../processed-v3/"
    os.makedirs(output_directory_for_save, exist_ok=True) # Ensure directory exists
    
    final_output_path = os.path.join(output_directory_for_save, final_occurrence_filename)

    try:
        occ_df_final_output.to_csv(final_output_path, index=False, na_rep='') 
        # The `current_run_type` variable is defined in the previous cell and is used here.
        print(f"\n💾 Taxonomically updated {current_run_type} occurrence file '{final_occurrence_filename}' saved to '{final_output_path}' with {len(occ_df_final_output)} records.")
        print(f"  Final columns written ({len(occ_df_final_output.columns)} total): {occ_df_final_output.columns.tolist()}")
        
        print(f"\n👀 Preview of final taxonomically updated {current_run_type} occurrence data (first 5 rows, selected columns):")
        preview_cols_final = ['eventID', 'occurrenceID', 'verbatimIdentification', 
                              'scientificName', 'scientificNameID', 'taxonRank', 'nameAccordingTo'] + DWC_RANKS_STD
        display_cols_final = [col for col in preview_cols_final if col in occ_df_final_output.columns]
        display(occ_df_final_output[display_cols_final].head())

    except Exception as e:
        print(f"  ❌ ERROR saving final taxonomically updated {current_run_type} occurrence file: {str(e)}")
        import traceback
        print(traceback.format_exc())

Starting post-matching processing.
  Post-processing (WoRMS): Found 10864 records with 'Eukaryota'-only verbatim strings that matched to high-level Eukaryota or were already 'incertae sedis'. Standardizing to 'incertae sedis'.
  Removed temporary columns: ['assay_name', 'match_type_debug']

💾 Taxonomically updated FULL DATASET occurrence file 'occurrence_worms_matched.csv' saved to '../processed-v3/occurrence_worms_matched.csv' with 312214 records.
  Final columns written (35 total): ['eventID', 'organismQuantity', 'occurrenceID', 'verbatimIdentification', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'scientificName', 'scientificNameID', 'taxonRank', 'identificationRemarks', 'taxonID', 'basisOfRecord', 'nameAccordingTo', 'organismQuantityType', 'recordedBy', 'materialSampleID', 'sampleSizeValue', 'sampleSizeUnit', 'associatedSequences', 'locationID', 'eventDate', 'minimumDepthInMeters', 'maximumDepthInMeters', 'locality', 'decimalLatitude', 'decimalLongitude', '

,eventID,occurrenceID,verbatimIdentification,scientificName,scientificNameID,taxonRank,nameAccordingTo,kingdom,phylum,class,order,family,genus,species
0,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_36aa75f9b28f5f831c2d631ba65c2bcb,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda,Crustacea,urn:lsid:marinespecies.org:taxname:1066,Subphylum,WoRMS,Animalia,Arthropoda,<NA>,<NA>,<NA>,<NA>,<NA>
1,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_4e38e8ced9070952b314e1880bede1ca,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Clausocalanus;Clausocalanus_furcatus;,Clausocalanus furcatus,urn:lsid:marinespecies.org:taxname:104503,Species,WoRMS,Animalia,Arthropoda,Copepoda,Calanoida,Clausocalanidae,Clausocalanus,<NA>
2,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_2a31e5c01634165da99e7381279baa75,Eukaryota;Obazoa;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Oithona;Oithona_sp.;,Oithona,urn:lsid:marinespecies.org:taxname:106485,Genus,WoRMS,Animalia,Arthropoda,Copepoda,Cyclopoida,Oithonidae,Oithona,<NA>
3,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_ecee60339b2fb88ea6d1c8d18054bed4,Eukaryota;TSAR;Alveolata;Dinoflagellata;Dinophyceae,Dinophyceae,urn:lsid:marinespecies.org:taxname:19542,Class,WoRMS,Chromista,Myzozoa,Dinophyceae,<NA>,<NA>,<NA>,<NA>
4,GOMECC18S_Plate4_53,GOMECC18S_Plate4_53_occ_fa1f1a97dd4ae7c826009186bad26384,Eukaryota;TSAR;Alveolata;Dinoflagellata;Dinophyceae;Gymnodiniales;Gymnodiniaceae,Gymnodiniaceae,urn:lsid:marinespecies.org:taxname:109410,Family,WoRMS,Chromista,Myzozoa,Dinophyceae,Gymnodiniales,Gymnodiniaceae,<NA>,<NA>


## Create DNA Derived Extension

First we clearly define the desired columns for the DNA Derived Extension output file:

In [34]:
# Define desired columns for DNA derived extension in output order
DESIRED_DNA_DERIVED_COLUMNS = [
    'eventID', 'source_mat_id', 'env_broad_scale', 'env_local_scale', 'env_medium', 
    'samp_vol_we_dna_ext', 'samp_collect_device', 'samp_mat_process', 'size_frac', 
    'concentration', 'lib_layout', 'seq_meth', 'nucl_acid_ext', 'target_gene', 
    'target_subfragment', 'pcr_primer_forward', 'pcr_primer_reverse', 
    'pcr_primer_name_forward', 'pcr_primer_name_reverse', 'pcr_primer_reference', 
    'pcr_cond', 'nucl_acid_amp', 'ampliconSize', 'otu_seq_comp_appr', 'otu_db', 
    'occurrenceID', 'DNA_sequence', 'concentrationUnit', 'otu_class_appr'
]

print(f"DNA derived extension will have {len(DESIRED_DNA_DERIVED_COLUMNS)} columns")

DNA derived extension will have 29 columns


Create foundation from occurrence core

In [35]:
# Start with the occurrence core data (occ_all_final_output) 
# Get the essential columns for DNA derived extension, and KEEP the correct assay_name column
print(f"Starting with the Occurrence Core's shape: {occ_all_final_output.shape}")

# Create foundation with the columns we need, including the correct assay_name
dna_derived_base = occ_all_final_output[['eventID', 'parentEventID', 'occurrenceID', 'materialSampleID', 'taxonID', 'assay_name']].copy()

# Rename materialSampleID to source_mat_id for final output
dna_derived_base = dna_derived_base.rename(columns={'materialSampleID': 'source_mat_id'})

# Extract featureID from taxonID (format: 'ASV:<featureid>')
dna_derived_base['featureID'] = dna_derived_base['taxonID'].str.replace('ASV:', '')

print(f"DNA derived base shape: {dna_derived_base.shape}")
print(f"Columns in base: {list(dna_derived_base.columns)}")

Starting with the Occurrence Core's shape: (312214, 36)
DNA derived base shape: (312214, 7)
Columns in base: ['eventID', 'parentEventID', 'occurrenceID', 'source_mat_id', 'taxonID', 'assay_name', 'featureID']


Merge with sampleMetadata to get sample collection information.

In [36]:
# Merge with sample metadata to get environmental and sample processing data
# Merge on: parentEventID (from dna_derived_base) = samp_name (from data['sampleMetadata'])

print(f"Before merge - dna_derived_base shape: {dna_derived_base.shape}")
print(f"data['sampleMetadata'] shape: {data['sampleMetadata'].shape}")

# Perform the merge. Pandas will add suffixes _x and _y for conflicting columns like 'assay_name'.
dna_derived_with_sample = dna_derived_base.merge(
    data['sampleMetadata'], 
    left_on='parentEventID', 
    right_on='samp_name', 
    how='left'
)

# FIX: The correct assay_name (from dna_derived_base) is now 'assay_name_x'.
# We rename it back to 'assay_name' and drop the useless one from the merge ('assay_name_y').
if 'assay_name_x' in dna_derived_with_sample.columns:
    dna_derived_with_sample.rename(columns={'assay_name_x': 'assay_name'}, inplace=True)
    if 'assay_name_y' in dna_derived_with_sample.columns:
        dna_derived_with_sample.drop(columns=['assay_name_y'], inplace=True)

print(f"\nAfter merge with sample metadata - shape: {dna_derived_with_sample.shape}")
dna_derived_with_sample.head()

Before merge - dna_derived_base shape: (312214, 7)
data['sampleMetadata'] shape: (472, 86)

After merge with sample metadata - shape: (312214, 92)


,eventID,parentEventID,occurrenceID,source_mat_id,taxonID,assay_name,featureID,samp_name,samp_category,materialSampleID,decimalLongitude,decimalLatitude,verbatimSRS,geo_loc_name,eventDate,env_broad_scale,env_local_scale,env_medium,habitat_natural_artificial_0_1,samp_collect_method,samp_collect_device,samp_size,samp_size_unit,samp_store_temp,samp_store_sol,...,omega_arag,pco2,phosphate,phosphate_unit,pressure,pressure_unit,silicate,silicate_unit,tot_alkalinity,tot_alkalinity_unit,transmittance,serial_number,line_id,station_id,ctd_cast_number,ctd_bottle_number,replicate_number,extract_id,extract_plate,extract_well_number,extract_well_position,biosample_accession,organism,dna_yield,dna_yield_unit
0,GOMECC18S_Plate4_53,GOMECC4_27N_Sta1_DCM_A,GOMECC18S_Plate4_53_occ_36aa75f9b28f5f831c2d631ba65c2bcb,GOMECC4_27N_Sta1_DCM,ASV:36aa75f9b28f5f831c2d631ba65c2bcb,ssu18sv9-emp,36aa75f9b28f5f831c2d631ba65c2bcb,GOMECC4_27N_Sta1_DCM_A,sample,GOMECC4_27N_Sta1_DCM,-79.618,26.997,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,Niskin bottle on CTD rosette,1540,mL,-80,none,...,3.805,423,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,GOMECC4_004,27N,Sta1,not provided,14,A,Plate4_53,GOMECC2021_Plate4,53,E7,SAMN37516094,seawater metagenome,223.5,ng
1,GOMECC18S_Plate4_53,GOMECC4_27N_Sta1_DCM_A,GOMECC18S_Plate4_53_occ_4e38e8ced9070952b314e1880bede1ca,GOMECC4_27N_Sta1_DCM,ASV:4e38e8ced9070952b314e1880bede1ca,ssu18sv9-emp,4e38e8ced9070952b314e1880bede1ca,GOMECC4_27N_Sta1_DCM_A,sample,GOMECC4_27N_Sta1_DCM,-79.618,26.997,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,Niskin bottle on CTD rosette,1540,mL,-80,none,...,3.805,423,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,GOMECC4_004,27N,Sta1,not provided,14,A,Plate4_53,GOMECC2021_Plate4,53,E7,SAMN37516094,seawater metagenome,223.5,ng
2,GOMECC18S_Plate4_53,GOMECC4_27N_Sta1_DCM_A,GOMECC18S_Plate4_53_occ_2a31e5c01634165da99e7381279baa75,GOMECC4_27N_Sta1_DCM,ASV:2a31e5c01634165da99e7381279baa75,ssu18sv9-emp,2a31e5c01634165da99e7381279baa75,GOMECC4_27N_Sta1_DCM_A,sample,GOMECC4_27N_Sta1_DCM,-79.618,26.997,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,Niskin bottle on CTD rosette,1540,mL,-80,none,...,3.805,423,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,GOMECC4_004,27N,Sta1,not provided,14,A,Plate4_53,GOMECC2021_Plate4,53,E7,SAMN37516094,seawater metagenome,223.5,ng
3,GOMECC18S_Plate4_53,GOMECC4_27N_Sta1_DCM_A,GOMECC18S_Plate4_53_occ_ecee60339b2fb88ea6d1c8d18054bed4,GOMECC4_27N_Sta1_DCM,ASV:ecee60339b2fb88ea6d1c8d18054bed4,ssu18sv9-emp,ecee60339b2fb88ea6d1c8d18054bed4,GOMECC4_27N_Sta1_DCM_A,sample,GOMECC4_27N_Sta1_DCM,-79.618,26.997,WGS84,"USA: Atlantic Ocean, east of Florida (27 N)",2021-09-14T11:00-04:00,marine biome [ENVO:00000447],marine photic zone [ENVO:00000209],sea water [ENVO:00002149],0,https://zenodo.org/records/14224755 (v1.1.0) protocol_sampling_sterivex_dry.md,Niskin bottle on CTD rosette,1540,mL,-80,none,...,3.805,423,0.0517,µmol/kg,49,dbar,1.05635,µmol/kg,2371,µmol/kg,4.665,GOMECC4_004,27N,Sta1,not provided,14,A,Plate4_53,GOMECC2021_Plate4,53,E7,SAMN37516094,seawater metagenome,223.5,ng
4,GOMECC18S_Plate4_53,GOMECC4_27N_Sta1_DCM_A,GOMECC18S_Plate4_53_occ_fa1f1a97dd4ae7c826009186bad26384,GOMECC4_27N_Sta1_DCM,ASV:fa1f1a97dd4ae7c826009186bad26384,ssu18sv9-emp,fa1f1a97dd4ae7c826009186bad26384,GOMECC4_27N_Sta1_DCM_A,sample,GOMECC4_27N_Sta1_DCM,-79.618,26.997,WGS84,"USA: Atlantic 

Add projectMetadata fields with proper project_level logic

In [37]:
# Handle projectMetadata in long format using the now-correct assay_name column

print(f"Current shape: {dna_derived_with_sample.shape}")
print("All assays in this submission:", dna_derived_with_sample['assay_name'].unique())
print("\n")

# Fields to populate from projectMetadata
project_fields_needed = [
    'lib_layout', 'instrument', 'target_gene', 'target_subfragment', 
    'pcr_primer_forward', 'pcr_primer_reverse', 'pcr_primer_name_forward', 
    'pcr_primer_name_reverse', 'pcr_primer_reference_forward', 'pcr_cond', 
    'nucl_acid_amp', 'ampliconSize'
]

# For each field, add it to our dataframe
print("Processing projectMetadata fields:")
for field in project_fields_needed:
    print(f"{field}")
    
    field_row = data['projectMetadata'][data['projectMetadata']['term_name'] == field]
    
    if not field_row.empty:
        field_row = field_row.iloc[0]
        project_level_val = field_row.get('project_level')
        
        # Check if a valid project_level value exists
        if pd.notna(project_level_val) and str(project_level_val).strip() and str(project_level_val).lower() != 'nan':
            dna_derived_with_sample[field] = project_level_val
            print(f"  -> Assigned project_level value '{project_level_val}' to all rows.")
        else:
            # Use assay-specific values
            print(f"  -> Using assay-specific values:")
            
            # Create a dictionary mapping each assay name to its corresponding value for the current field
            assay_value_map = {assay: field_row.get(assay) for assay in dna_derived_with_sample['assay_name'].unique()}
            
            # Use the .map() function for an efficient lookup based on the now-correct 'assay_name' column
            dna_derived_with_sample[field] = dna_derived_with_sample['assay_name'].map(assay_value_map)
            
            # Show the unique values that were assigned to the column for verification
            assigned_values = dna_derived_with_sample[field].dropna().unique()
            print(f"  -> Assigned values: {list(assigned_values)}")
    else:
        print(f"WARNING: {field} not found in projectMetadata, column will be empty.")
        dna_derived_with_sample[field] = None

print("-" * 50)
print(f"Finished processing. Final shape: {dna_derived_with_sample.shape}")
print("Columns added/updated:", [col for col in project_fields_needed if col in dna_derived_with_sample.columns])

Current shape: (312214, 92)
All assays in this submission: ['ssu18sv9-emp' 'ssu16sv4v5-emp']


Processing projectMetadata fields:
lib_layout
  -> Assigned project_level value 'paired end' to all rows.
instrument
  -> Using assay-specific values:
  -> Assigned values: ['Illumina MiSeq [OBI_0002003]']
target_gene
  -> Using assay-specific values:
  -> Assigned values: ['18S rRNA (SSU eukaryote)', '16S rRNA (SSU prokaryote)']
target_subfragment
  -> Using assay-specific values:
  -> Assigned values: ['V9', 'V4-V5']
pcr_primer_forward
  -> Using assay-specific values:
  -> Assigned values: ['GTACACACCGCCCGTC', 'GTGYCAGCMGCCGCGGTAA']
pcr_primer_reverse
  -> Using assay-specific values:
  -> Assigned values: ['TGATCCTTCTGCAGGTTCACCTAC', 'CCGYCAATTYMTTTRAGTTT']
pcr_primer_name_forward
  -> Using assay-specific values:
  -> Assigned values: ['1391f', '515F-Y']
pcr_primer_name_reverse
  -> Using assay-specific values:
  -> Assigned values: ['EukBr', '926R']
pcr_primer_reference_forward
  -> Usi

Add analysisMetadata fields using occurrence mapping

In [38]:
# Use the individual dataframes that were combined to create the occurrence core
# Map them to analysis runs using the order from params['datafiles']

print("Available variables to check for analysis mapping:")
print("- all_processed_occurrence_dfs:", 'all_processed_occurrence_dfs' in locals())

if 'all_processed_occurrence_dfs' in locals():
    analysis_run_order = list(params['datafiles'].keys())
    print(f"Number of dataframes: {len(all_processed_occurrence_dfs)}")
    print(f"Analysis run order: {analysis_run_order}")
    
    # Create a mapping of row indices to analysis runs
    analysis_fields_needed = ['otu_seq_comp_appr', 'otu_db', 'otu_clust_tool']
    
    # Initialize columns
    for field in analysis_fields_needed:
        dna_derived_with_sample[field] = None
    
    current_row_start = 0
    for i, df in enumerate(all_processed_occurrence_dfs):
        if i >= len(analysis_run_order):
            print(f"Warning: More dataframes than analysis runs defined")
            break
            
        analysis_run_name = analysis_run_order[i]
        print(f"DataFrame {i}: {len(df)} rows -> {analysis_run_name}")
        
        # Find which assay this analysis run belongs to by checking the data structure
        assay_key = None
        for assay_name, analysis_runs in data['analysis_data_by_assay'].items():
            if analysis_run_name in analysis_runs:
                assay_key = assay_name
                break
        
        if not assay_key:
            print(f"  Warning: Could not find assay for {analysis_run_name}")
            current_row_start += len(df)
            continue
            
        current_row_end = current_row_start + len(df)
        print(f"  Rows {current_row_start} to {current_row_end-1} (assay: {assay_key})")
        
        # Get analysis metadata for this run
        if analysis_run_name in data['analysis_data_by_assay'][assay_key]:
            analysis_df = data['analysis_data_by_assay'][assay_key][analysis_run_name]
            
            for field in analysis_fields_needed:
                field_row = analysis_df[analysis_df['term_name'] == field]
                if not field_row.empty:
                    field_value = field_row.iloc[0]['values']
                    # Apply to the specific row range
                    dna_derived_with_sample.loc[current_row_start:current_row_end-1, field] = field_value
                    print(f"    {field}: {field_value}")
        
        current_row_start = current_row_end
    
    print(f"\nAfter adding analysis fields - shape: {dna_derived_with_sample.shape}")
else:
    print("Required variables not available")

Available variables to check for analysis mapping:
- all_processed_occurrence_dfs: True
Number of dataframes: 3
Analysis run order: ['gomecc4_18s_p1-6_v2024.10_241122', 'gomecc4_16s_p3-6_v2024.10_241122', 'gomecc4_16s_p1-2_v2024.10_241122']
DataFrame 0: 147083 rows -> gomecc4_18s_p1-6_v2024.10_241122
  Rows 0 to 147082 (assay: ssu18sv9-emp)
    otu_seq_comp_appr: qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1
    otu_db: PR2 v5.0.1; V9 1391f-1510r region; 10.5281/zenodo.8392706
    otu_clust_tool: Tourmaline; qiime2-2021.2; dada2
DataFrame 1: 118671 rows -> gomecc4_16s_p3-6_v2024.10_241122
  Rows 147083 to 265753 (assay: ssu16sv4v5-emp)
    otu_seq_comp_appr: qiime2-2021.2; naive bayes classifier; scikit-learn 0.23.1
    otu_db: Silva SSU Ref NR 99 v138.1; 515f-926r region; 10.5281/zenodo.8392695
    otu_clust_tool: Tourmaline; qiime2-2021.2; dada2
DataFrame 2: 46460 rows -> gomecc4_16s_p1-2_v2024.10_241122
  Rows 265754 to 312213 (assay: ssu16sv4v5-emp)
    otu_seq_comp_ap

Merge DNA sequences using featureID

In [39]:
# Merge DNA sequences from raw taxonomy files - fix to prevent row explosion
print("Merging DNA sequences from raw taxonomy files...")

# Use the raw_data_tables that were loaded earlier
all_tax_data = []
if 'raw_data_tables' in locals() and raw_data_tables:
    for analysis_run_name, data_dict in raw_data_tables.items():
        if 'taxonomy' in data_dict:
            tax_data = data_dict['taxonomy'].copy()
            tax_data['analysis_run'] = analysis_run_name  # Track source
            all_tax_data.append(tax_data)
            print(f"Found taxonomy data for {analysis_run_name}: {tax_data.shape}")

if all_tax_data:
    combined_tax_data = pd.concat(all_tax_data, ignore_index=True)
    print(f"Combined taxonomy data shape: {combined_tax_data.shape}")
    
    # Find the column containing DNA sequences and feature IDs
    feature_col = None
    dna_col = None
    
    for col in combined_tax_data.columns:
        if 'featureid' in col.lower() or col.lower() == 'otu id':
            feature_col = col
        elif 'dna_sequence' in col.lower() or 'sequence' in col.lower():
            dna_col = col
    
    print(f"Using feature column: {feature_col}")
    print(f"Using DNA sequence column: {dna_col}")
    print(f"Available columns: {list(combined_tax_data.columns)}")
    
    if feature_col and dna_col:
        # CRITICAL FIX: Remove duplicates from taxonomy data BEFORE merging
        seq_data = combined_tax_data[[feature_col, dna_col]].drop_duplicates(subset=[feature_col])
        print(f"After removing duplicate featureIDs: {seq_data.shape}")
        
        # Now merge (should be one-to-one)
        print(f"Before DNA merge: {dna_derived_with_sample.shape}")
        dna_derived_with_sequences = dna_derived_with_sample.merge(
            seq_data, 
            left_on='featureID', 
            right_on=feature_col, 
            how='left'
        )
        
        # Rename the DNA sequence column to match our desired output
        dna_derived_with_sequences = dna_derived_with_sequences.rename(columns={dna_col: 'DNA_sequence'})
        
        print(f"After merging DNA sequences - shape: {dna_derived_with_sequences.shape}")
        print(f"Rows missing DNA sequences: {dna_derived_with_sequences['DNA_sequence'].isna().sum()}/{len(dna_derived_with_sequences)}")
        
        # Drop the extra feature column from the merge if it's different from our featureID
        if feature_col in dna_derived_with_sequences.columns and feature_col != 'featureID':
            dna_derived_with_sequences = dna_derived_with_sequences.drop(columns=[feature_col])
    else:
        print("Could not find required columns for DNA sequence merging")
        print("Available columns in taxonomy data:", list(combined_tax_data.columns))
        dna_derived_with_sequences = dna_derived_with_sample.copy()
        dna_derived_with_sequences['DNA_sequence'] = None
else:
    print("No taxonomy data found in raw_data_tables")
    dna_derived_with_sequences = dna_derived_with_sample.copy()
    dna_derived_with_sequences['DNA_sequence'] = None

Merging DNA sequences from raw taxonomy files...
Found taxonomy data for gomecc4_18s_p1-6_v2024.10_241122: (24473, 15)
Found taxonomy data for gomecc4_16s_p3-6_v2024.10_241122: (49523, 13)
Found taxonomy data for gomecc4_16s_p1-2_v2024.10_241122: (19540, 13)
Combined taxonomy data shape: (93536, 16)
Using feature column: featureid
Using DNA sequence column: dna_sequence
Available columns: ['featureid', 'dna_sequence', 'taxonomy', 'verbatimIdentification', 'kingdom', 'supergroup', 'division', 'subdivision', 'class', 'order', 'family', 'genus', 'species', 'Confidence', 'analysis_run', 'phylum']
After removing duplicate featureIDs: (89631, 2)
Before DNA merge: (312214, 107)
After merging DNA sequences - shape: (312214, 109)
Rows missing DNA sequences: 0/312214


Apply final mapping and create output

In [40]:
# Apply Darwin Core field mappings and create final output
print("Applying Darwin Core mappings and creating final DNA derived extension...")

# The dwc_data is a dictionary containing DataFrames
dna_derived_mapping = dwc_data['dnaDerived']
print(f"Available Darwin Core mappings for dnaDerived: {len(dna_derived_mapping)}")
print("\n")

# Create mapping dictionary from FAIRe names to Darwin Core names
# Note: Darwin Core terms are the INDEX, FAIRe terms are in 'FAIRe_term' column
field_mapping = {}
for dwc_name, row in dna_derived_mapping.iterrows():
    faire_name = row['FAIRe_term']  # Changed from 'FAIRe_name' to 'FAIRe_term'
    if pd.notna(faire_name) and pd.notna(dwc_name):
        field_mapping[faire_name] = dwc_name

print(f"Field mappings to apply: {len(field_mapping)}")

# Apply the mappings
dna_derived_df_final = dna_derived_with_sequences.copy()
for faire_name, dwc_name in field_mapping.items():
    if faire_name in dna_derived_df_final.columns:
        dna_derived_df_final = dna_derived_df_final.rename(columns={faire_name: dwc_name})
        print(f"Mapped {faire_name} → {dwc_name}")

# Ensure we have all required columns in the correct order
final_columns = []
for col in DESIRED_DNA_DERIVED_COLUMNS:
    if col in dna_derived_df_final.columns:
        final_columns.append(col)
    else:
        print(f"WARNING: Missing required column: {col}")
        dna_derived_df_final[col] = pd.NA
        final_columns.append(col)

# Select only the required columns in the correct order
dna_derived_extension = dna_derived_df_final[final_columns]

print(f"\nFinal DNA derived extension shape: {dna_derived_extension.shape}")
print(f"Expected rows (should match occurrence core): {len(occ_all_final_output)}")
print(f"Row count match: {len(dna_derived_extension) == len(occ_all_final_output)}")

print(f"\nColumn order verification:")
for i, col in enumerate(dna_derived_extension.columns):
    print(f"{i+1:2d}. {col}")

print(f"\nData completeness summary:")
for col in dna_derived_extension.columns:
    non_null = dna_derived_extension[col].notna().sum()
    pct = (non_null / len(dna_derived_extension)) * 100
    print(f"  {col}: {non_null:,} ({pct:.1f}%)")

# Display sample of final data
print(f"\nSample of final DNA derived extension:")
print(dna_derived_extension.head())

Applying Darwin Core mappings and creating final DNA derived extension...
Available Darwin Core mappings for dnaDerived: 27


Field mappings to apply: 27
Mapped env_broad_scale → env_broad_scale
Mapped env_local_scale → env_local_scale
Mapped env_medium → env_medium
Mapped samp_collect_device → samp_collect_device
Mapped samp_mat_process → samp_mat_process
Mapped size_frac → size_frac
Mapped samp_vol_we_dna_ext → samp_vol_we_dna_ext
Mapped nucl_acid_ext → nucl_acid_ext
Mapped concentration → concentration
Mapped concentration_unit → concentrationUnit
Mapped target_gene → target_gene
Mapped target_subfragment → target_subfragment
Mapped ampliconSize → ampliconSize
Mapped pcr_primer_forward → pcr_primer_forward
Mapped pcr_primer_reverse → pcr_primer_reverse
Mapped pcr_primer_name_forward → pcr_primer_name_forward
Mapped pcr_primer_name_reverse → pcr_primer_name_reverse
Mapped pcr_primer_reference_forward → pcr_primer_reference
Mapped nucl_acid_amp → nucl_acid_amp
Mapped pcr_cond → pcr_co

Save DNA Derived Extension to a CSV. You are now ready to submit to OBIS/GBIF!

In [41]:
# Save DNA derived extension to CSV file
print("Saving DNA derived extension to CSV...")

# Save to the processed directory (same as occurrence.csv)
output_path = "../processed-v3/dna_derived_extension.csv"
dna_derived_extension.to_csv(output_path, index=False)

print(f"DNA derived extension saved to: {output_path}")
print(f"File contains {len(dna_derived_extension)} rows and {len(dna_derived_extension.columns)} columns")

# Verify the file was created
import os
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024*1024)  # Size in MB
    print(f"File size: {file_size:.2f} MB")
    print("✓ DNA derived extension file successfully created!")
else:
    print("❌ Error: File was not created")

Saving DNA derived extension to CSV...
DNA derived extension saved to: ../processed-v3/dna_derived_extension.csv
File contains 312214 rows and 29 columns
File size: 329.31 MB
✓ DNA derived extension file successfully created!
